In [1]:
import csv
import math
import numpy as np
import pandas as pd
import pingouin as pg
import researchpy as rp
# from scipy import optimize
# from scipy import stats  # non-ANOVA, see https://docs.scipy.org/doc/scipy/reference/stats.html
import seaborn as sns
import statsmodels.stats.multicomp as mc
import statsmodels.api as sm
from statsmodels.formula.api import ols

%matplotlib notebook        
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D   # plot in 3D
from matplotlib.patches import Polygon
from matplotlib.ticker import FormatStrFormatter

/home/natapueyo/miniconda3/envs/wall_2020/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# display options

# from https://stackoverflow.com/questions/21008858/formatting-floats-in-a-numpy-array
# function to display floats in numpy arrays to 3 decimals of precision (does not affect computation)
float_formatter = "{:.3f}".format
# print all numpy array floats to 3 decimal precision
np.set_printoptions(formatter={'float_kind':float_formatter})

# Statistical tests of fruit data such as determining if the distribution of the z-coordinates of the fruits matches the Uniform or Normal distributions

## Obtain the data from the 2022 fruit data CSV files and break it into segments

In [3]:
# ## Obtain fruit data from the 2022 Field Trip
# n_files = 15

# length_segment = 3.5
# max_length     = 14.97 # ish

# n_seg = math.floor(max_length/length_segment)+1
# print('there should be %d segments' %n_seg)

# seg_n = np.arange(n_seg)
# print('so the array of segments is', seg_n, '\n')

# y_lim          = np.array([-length_segment, length_segment])

# segment_x_list = list()
# segment_y_list = list()
# segment_z_list = list()

# # test for the max value in x, y, and z for all segments
# seg_max_x = 0
# seg_min_x = 1000
# seg_max_y = 0
# seg_min_y = 1000
# seg_max_z = 0
# seg_min_z = 1000

# segment_N_list = list() # saves the number of fruits in each segment

# csv_folder = './TREE_FRUIT_DATA/apple_data_2022/'

# for run_n in range(n_files):
#     if run_n <= 4:
#         run_files = '01_42_4_' + str(run_n+2) + '.csv'
#     elif run_n <= 9:
#         run_files = '46_42_1_' + str(run_n-4) + '.csv'
#     elif run_n <= 13:
#         run_files = '51_42_2_' + str(run_n-9) + '.csv'
#     elif run_n <= 16:
#         run_files = '56_42_3_' + str(run_n-13) + '.csv'
#     elif run_n <= 17:
#         run_files = '30_38_2_1.csv'
# #     if run_n <= 4:
# #         run_files = '01_42_4_' + str(run_n+2) + '.csv'
# #     elif run_n <= 9:
# #         run_files = '46_42_1_' + str(run_n-4) + '.csv'
# #     elif run_n <= 13:
# #         run_files = '51_42_2_' + str(run_n-9) + '.csv'
# #     elif run_n <= 16:
# #         run_files = '56_42_3_' + str(run_n-13) + '.csv'
# #     print('CSV file number:', run_n)
# #     if run_n <= 5:
# #         run_files = '01_42_4_' + str(run_n+1) + '.csv'
# #     elif run_n <= 10:
# #         run_files = '46_42_1_' + str(run_n-5) + '.csv'
# #     elif run_n <= 14:
# #         run_files = '51_42_2_' + str(run_n-10) + '.csv'
# #     elif run_n <= 17:
# #         run_files = '56_42_3_' + str(run_n-14) + '.csv'
# #     elif run_n <= 18:
# #         run_files = '30_38_2_1.csv'
# #         # only one file of this denomination
# #     elif run_n <= 19:
# #         # only one file of this denomination
# #         run_files = '41_42_0_1.csv'
# #     elif run_n <= 24:
# #         run_files = '41_42_12_' + str(run_n-19) + '.csv'
# #     elif run_n <= 28:
# #         run_files = '50_38_6_' + str(run_n-24) + '.csv'
# #     elif run_n <= 31:
# #         run_files = '06_42_5_' + str(run_n-28) + '.csv'
# #     else: 
# #         print('WARNING: There are no more files in the 2022 dataset to run. Defaulting to the last file.')
# #         run_files = '06_42_5_4.csv'
    
#     # name of the CSV file that will be opened
#     csv_file = csv_folder + run_files
#     print('csv file', csv_file)
    
#     # init the number of fruits and the list of fruits for this segment
#     n_fruit_file = 0
#     x_list_file  = list()
#     y_list_file  = list()  # need the y-coordinates to separate fruits into their respective segments
#     z_list_file  = list()

#     # open the file once and later separate into segments (don't have to open it for every segment)
#     with open(csv_file) as csvfile:
#         reader = csv.reader(csvfile, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC)
#         for row in reader:
#             # the 2022 data is the only data set that has different x, y, z definitions than the rest of the project
#             y_list_file.append(row[0]) # length along orchard row
#             z_list_file.append(row[1]) # height
#             x_list_file.append(row[2]) # height
            
#             n_fruit_file += 1

#     # turn the file's list of z-coordinates into an array for easier work
#     x = np.array(x_list_file)
#     y = np.array(y_list_file)
#     z = np.array(z_list_file)
    
#     # remove any negative values and start at a min value for the whole orchard segment 
#     x_translate = np.amin(x) - 0.01
#     x = x - x_translate
#     y_translate = np.amin(y) - 0.01
#     y = y - y_translate
#     z_translate = np.amin(z) - 0.01
#     z = z - z_translate
    
#     if seg_max_x < np.amax(x):
#         seg_max_x = np.amax(x)
#     if seg_min_x > np.amin(x):
#         seg_min_x = np.amin(x)
        
#     if seg_max_y < np.amax(y):
#         seg_max_y = np.amax(y)
#     if seg_min_y > np.amin(y):
#         seg_min_y = np.amin(y)
        
#     if seg_max_z < np.amax(z):
#         seg_max_z = np.amax(z)
#     if seg_min_z > np.amin(z):
#         seg_min_z = np.amin(z)
    
#     for seg in seg_n:
#         # reset y_seg and z_seg to contain all fruits from the CSV (will be cleaned up in a bit)
#         x_seg = np.copy(x)
#         y_seg = np.copy(y)
#         z_seg = np.copy(z)
        
#         y_limit = [seg*y_lim[1], (seg+1)*y_lim[1]] # sets the segments starts and ends at based on the segment number (keep things consistent between runs/datasets)
# #         print('*************** PRINT y_limits ******************', y_limit)
        
#         # only use the fruits within this segment
#         index_out_of_y_bounds = np.where((y <= y_limit[0]) | (y >= y_limit[1]))
#         # remove any fruits that fell outside of the segment
#         x_seg = np.delete(x_seg, index_out_of_y_bounds[0])
#         y_seg = np.delete(y_seg, index_out_of_y_bounds[0])
#         z_seg = np.delete(z_seg, index_out_of_y_bounds[0])
        
#         if len(z_seg) > 0:
#             segment_y_list.append(y_seg)
#             segment_z_list.append(z_seg)

#             segment_N_list.append(len(z_seg))
            
#             if len(z_seg) > 190:
#                 ax = plt.figure().add_subplot(projection='3d')
#                 ax.scatter(x_seg, y_seg, z_seg, color='r')
#                 ax.set_xlabel('distance into canopy [m]')
#                 ax.set_ylabel('distance along orchard row [m]')
#                 ax.set_zlabel('distance height [m]')
#                 ax.view_init(elev=12, azim=-166)
# #                 file_name = './plots/minfpt_cr33_bestv.pdf'
# #                 print('Saving plot of FPT and FPE vs density in', file_name)
# #                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
#                 plt.show()
                
            
# #             if run_n == 1 and seg == 3: # index of seg starts at 0
# #                 # min fpt for both partitions, bestV, CR33
# #                 print('for CR33, bestV the min FPT happens in run = 1, seg = 2')
# #                 print('     N = %d\n' %len(y_seg))
                
# #                 ax = plt.figure().add_subplot(projection='3d')
# #                 ax.scatter(x_seg, y_seg, z_seg, color='r')
# #                 ax.set_xlabel('distance into canopy [m]')
# #                 ax.set_ylabel('distance along orchard row [m]')
# #                 ax.set_zlabel('distance height [m]')
# #                 ax.view_init(elev=12, azim=-166)
# #                 file_name = './plots/minfpt_cr33_bestv.pdf'
# #                 print('Saving plot of FPT and FPE vs density in', file_name)
# #                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
# #                 plt.show()

# #             elif run_n == 19 and seg == 1:
# #                 # max fpt for height partition, bestV, CR33
# #                 print('for CR33, bestV the max FPT for height happens in run = 19, seg = 1')
# #                 print('     N = %d\n' %len(y_seg))
                
# #                 ax = plt.figure().add_subplot(projection='3d')
# #                 ax.scatter(x_seg, y_seg, z_seg, color='r')
# #                 ax.set_xlabel('distance into canopy [m]')
# #                 ax.set_ylabel('distance along orchard row [m]')
# #                 ax.set_zlabel('distance height [m]')
# #                 ax.view_init(elev=12, azim=-166)
# #                 file_name = './plots/maxfpt_height_cr33_bestv.pdf'
# #                 print('Saving plot of FPT and FPE vs density in', file_name)
# #                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
# #                 plt.show()
                
# #             elif run_n == 30 and seg == 1:
# #                 # max fpt for fruit partition, bestV, CR33
# #                 print('for CR33, bestV the min FPT for fruit happens in run = 29, seg = 1')
# #                 print('     N = %d\n' %len(y_seg))
                
# #                 ax = plt.figure().add_subplot(projection='3d')
# #                 ax.scatter(x_seg, y_seg, z_seg, color='r')
# #                 ax.set_xlabel('distance into canopy [m]')
# #                 ax.set_ylabel('distance along orchard row [m]')
# #                 ax.set_zlabel('distance height [m]')
# #                 ax.view_init(elev=12, azim=-166)
# #                 file_name = './plots/maxfpt_fruit_cr33_bestv.pdf'
# #                 print('Saving plot of FPT and FPE vs density in', file_name)
# #                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
# #                 plt.show()
                
        
# # save the number of fruits as an array
# N_array    = np.array(segment_N_list)
# n_segments = len(N_array)

# print('there are %d segments with fruits in them\n' %n_segments)

# print('the max values for each axis are:\n        x:              y:              z:')
# print('         %3.2f            %3.2f           %3.2f\n' %(seg_max_x, seg_max_y, seg_max_z))
# print('the min values for each axis are:\n        x:              y:              z:')
# print('         %3.2f            %3.2f           %3.2f\n' %(seg_min_x, seg_min_y, seg_min_z))


# print('each segment contains', N_array, 'fruits')


there should be 5 segments
so the array of segments is [0 1 2 3 4] 

csv file ./TREE_FRUIT_DATA/apple_data_2022/01_42_4_2.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/01_42_4_3.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/01_42_4_4.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/01_42_4_5.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/01_42_4_6.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/46_42_1_1.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/46_42_1_2.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/46_42_1_3.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/46_42_1_4.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/46_42_1_5.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/51_42_2_1.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/51_42_2_2.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/51_42_2_3.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/51_42_2_4.csv
csv file ./TREE_FRUIT_DATA/apple_data_2022/56_42_3_1.csv
there are 54 segments with fruits in them

the max values for each axis are:

# Statistical analysis of the number of fruits in each segment

In [4]:
# # find outliers/skew in the data (number of fruits) using z-score
# N_mean = np.mean(N_array) 
# N_std = np.std(N_array) 
# print('mean of the fruit dataset is', N_mean, 'fruits') 
# print('std. deviation is', N_std, '\n') 

# threshold = 3
# outlier = [] 

# for i in range(len(N_array)):
#     z = (N_array[i] - N_mean) / N_std 
    
#     if z > threshold or z < -threshold: 
#         outlier.append(i) 
# print('indexes of the outliers in dataset:\n    ', outlier)
# print('values of the outliers:\n    ', N_array[outlier], '\n')

# # check the skew of the data
# res = stats.skewtest(N_array)
# print('check if the data is skewed enough to not fit a normal distribution:')
# print('      statistic=', res.statistic, ' pvalue=', res.pvalue, '\n')
# print('there is ', stats.skew(N_array),' skew')


# # counts, bins = np.histogram(N_array,10)
# # print('\nnp.histogram results: count', counts, ' and bins', bins)
# # plt.stairs(counts, bins)

# fig, ax = plt.subplots()
                       
# n, bins, patches = plt.hist(N_array, bins='auto')#, density=True)

# # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
# # ax.plot(bins, y, '--')

# ax.set_xlabel('Number of fruits')
# ax.set_ylabel('Counts')
# ax.set_title('Histogram of the number of fruits in each segment')
# # ax.set_title('Histogram of normal distribution sample: 'fr'$\mu={N_mean:.2f}$, $\sigma={N_std:.2f}$')

# plt.show()


## Uniform Distribution goodness of fit

In [5]:
# # read a CSV file of the input data and test for uniformity ( Kolmogorov-Smirnov test for goodness of fit) 
# # of the distribution of the fruits's z-coordinates along the z-axis for every segment

# # two flags to get a uniform and not uniform histogram
# uniform     = 0
# not_uniform = 0

# # count how many segments are uniform and how many are not 
# n_tested      = 0
# n_uniform     = 0
# n_not_uniform = 0

# # save the p-values to see the distribution of the p-vals from the goodness of fit test
# p_val_list  = list()

# for seg_i in range(n_segments):
#     print('Segment number %d' %seg_i)
    
#     z_seg = segment_z_list[seg_i] 
    
#     # check if there are enough fruits in this segment to run the test
#     if len(z_seg) >= 3:
#         n_tested += 1

#         # perform uniformity test
# #             rng = np.random.default_rng()
# #             res = stats.kstest(z_seg, 3.5*stats.uniform.rvs(size=1000, random_state=rng))
#         res = stats.cramervonmises(z_seg, stats.uniform(loc=0.0, scale=3.5).cdf)

# #             res = stats.shapiro(z_seg)
# #         print(f'Harvesting a segment of the CSV file between y-coordinates %3.3f and %3.3f m' %(y_limit[0], y_limit[1]))
#         print(f'the number of fruits in this test is %d\n' %len(z_seg))
#         print('Cramer von Mises test for goodness of fit results for this segment assuming $\alpha = 0.05')
#         print(f'        statistic=%3.6f        p-value=%3.6f' %(res.statistic, res.pvalue))

#         p_val_list.append(res.pvalue)

#         if res.pvalue > 0.05:
#             # cannot reject the null and the data is likely uniform
#             print('cannot reject the null; Probably uniform\n')
#             n_uniform += 1

#             # get a good "uniform distribution" graph
#             if res.pvalue > 0.05 and len(z_seg) >= 15 and uniform != 1:
#                 uniform = 1

#                 fig, ax = plt.subplots()

#                 n, bins, patches = plt.hist(z_seg, bins='auto')#, density=True)

#                 # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
#                 # ax.plot(bins, y, '--')

#                 ax.set_xlabel('Height (m)')
#                 ax.set_ylabel('Counts')
#                 ax.set_title('Histogram of uniform distribution sample')
#                 # ax.set_title('Histogram of uniform distribution sample: 'fr'$\mu={N_mean:.2f}$, $\sigma={N_std:.2f}$')

#                 plt.show()

#         else:
#             # we reject the null
#             print('reject the null; Probably NOT uniform\n')

#             n_not_uniform += 1

#             # get a good "NOT uniform distribution" graph
#             if res.pvalue < 0.001 and len(segment_z_list[seg_i]) >= 69 and not_uniform != 1:
#                 not_uniform = 1

#                 fig, ax = plt.subplots()

#                 n, bins, patches = plt.hist(segment_z_list[seg_i], bins='auto')#, density=True)

#                 # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
#                 # ax.plot(bins, y, '--')

#                 ax.set_xlabel('Height (m)')
#                 ax.set_ylabel('Counts')
#                 ax.set_title('Histogram of NON-uniform distribution sample')

#                 plt.show()

#     elif len(z_seg) > 0: 
# #         print(f'Harvesting a segment of the CSV file between y-coordinates %3.3f and %3.3f m' %(y_limit[0], y_limit[1]))
#         print('Kolmogorov-Smirnov test cannot run if N < 3?')
#         print(f'the number of fruits in this test is %d\n' %len(segment_z_list[seg_i]))

#     print('--------------------------------------------------------------------------------')

#     # get a couple of histograms

# print(f'total tested segments %d out of which \n         %d are uniform         %d are NOT uniform' %(n_tested, n_uniform, n_not_uniform))

# # make p_val_list an array so we can analyze it
# p_val_array = np.array(p_val_list)


# # plot the histogram of the p-values
# fig, ax = plt.subplots()
                       
# n, bins, patches = plt.hist(p_val_array, bins=15)#, density=True)

# # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
# # ax.plot(bins, y, '--')

# ax.set_xlabel('p-value')
# ax.set_ylabel('Counts')
# ax.set_title('Histogram of the p-values of the uniformity test of the segments, bins=15')

# plt.show()


## Normal Distribution goodness of fit

In [6]:
# # Use CSV data to test if the distribution of the fruits's z-coordinates fits a normal distribution
# # based on the Shapiro-Wilk goodness of fit test

# # two flags to get a uniform and not uniform histogram
# normal     = 0
# not_normal = 0

# # count how many segments are uniform and how many are not 
# n_tested     = 0
# n_normal     = 0
# n_not_normal = 0

# # save the p-values to see the distribution of the p-vals from the goodness of fit test
# p_val_list  = list()

# for seg_i in range(n_segments):
#     print('Segment number %d' %seg_i)
    
#     z_seg = segment_z_list[seg_i] 
    
#     # check if there are enough fruits in this segment to run the test
#     if len(z_seg) >= 3:
#         n_tested += 1
        
#         # perform normality test
#         res = stats.shapiro(z_seg)
# #         print(f'Harvesting a segment of the CSV file between y-coordinates %3.3f and %3.3f m' %(y_limit[0], y_limit[1]))
#         print(f'the number of fruits in this test is %d\n' %len(z_seg))
#         print('Shapiro-Wilk normality test results for this segment assuming $\alpha = 0.05')
#         print(f'        statistic=%3.6f        p-value=%3.6f' %(res.statistic, res.pvalue))

#         p_val_list.append(res.pvalue)

#         if res.pvalue > 0.05:
#             # do not reject the null and the data is likely normal
#             print('Probably normal\n')
#             n_normal += 1

#             # get a good "normal distribution" graph
#             if res.pvalue > 0.5 and len(z_seg) >= 65 and normal != 1:
#                 normal = 1

#                 fig, ax = plt.subplots()

#                 n, bins, patches = plt.hist(z_seg, bins='auto')#, density=True)

#                 # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
#                 # ax.plot(bins, y, '--')

#                 ax.set_xlabel('Height (m)')
#                 ax.set_ylabel('Counts')
#                 ax.set_title('Histogram of normal distribution sample')
#                 # ax.set_title('Histogram of normal distribution sample: 'fr'$\mu={N_mean:.2f}$, $\sigma={N_std:.2f}$')

#                 plt.show()

#         else:
#             # we reject the null
#             print('Probably NOT normal\n')

#             n_not_normal += 1

#             # get a good "NOT normal distribution" graph
#             if res.pvalue < 0.001 and len(z_seg) >= 65 and not_normal != 1:
#                 not_normal = 1

#                 fig, ax = plt.subplots()

#                 n, bins, patches = plt.hist(z_seg, bins='auto')#, density=True)

#                 # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
#                 # ax.plot(bins, y, '--')

#                 ax.set_xlabel('Height (m)')
#                 ax.set_ylabel('Counts')
#                 ax.set_title('Histogram of NON-normal distribution sample')

#                 plt.show()

#     elif len(z_seg) > 0: 
# #         print(f'Harvesting a segment of the CSV file between y-coordinates %3.3f and %3.3f m' %(y_limit[0], y_limit[1]))
#         print('Shapiro-Wilk normality test cannot run if N < 3')
#         print(f'the number of fruits in this test is %d\n' %len(z_seg))

#     print('--------------------------------------------------------------------------------')

#     # get a couple of histograms

# print(f'total tested segments %d out of which \n         %d are normal         %d are NOT normal' %(n_tested, n_normal, n_not_normal))

# # make p_val_list an array so we can analyze it
# p_val_array = np.array(p_val_list)


# # plot the histogram of the p-values
# fig, ax = plt.subplots()
                       
# n, bins, patches = plt.hist(p_val_array, bins=15)#, density=True)

# # y = ((1 / (np.sqrt(2 * np.pi) * N_mean)) * np.exp(-0.5 * (1 / N_mean * (bins - N_std))**2))
# # ax.plot(bins, y, '--')

# ax.set_xlabel('p-value')
# ax.set_ylabel('Counts')
# ax.set_title('Histogram of the p-values of the normality test of the segments, bins=15')

# plt.show()


In [12]:
## Obtain fruit data from the 2022 Field Trip
n_files = 32 

# length_segment = 3.5
# max_length     = 14.97 # ish

# n_seg = math.floor(max_length/length_segment)+1
# print('there should be %d segments' %n_seg)

# seg_n = np.arange(n_seg)
# print('so the array of segments is', seg_n, '\n')

# y_lim          = np.array([-length_segment, length_segment])

file_x_list = list()
file_y_list = list()
file_z_list = list()

# test for the max value in x, y, and z for all files
file_max_x = 0
file_min_x = 1000
file_max_y = 0
file_min_y = 1000
file_max_z = 0
file_min_z = 1000

sum_length = 0

file_N_list = list() # saves the number of fruits in each segment

csv_folder = './TREE_FRUIT_DATA/apple_data_2022/'

for run_n in range(n_files):
#     print('CSV file number:', run_n)
    if run_n <= 5:
        run_files = '01_42_4_' + str(run_n+1) + '.csv'
    elif run_n <= 10:
        run_files = '46_42_1_' + str(run_n-5) + '.csv'
    elif run_n <= 14:
        run_files = '51_42_2_' + str(run_n-10) + '.csv'
    elif run_n <= 17:
        run_files = '56_42_3_' + str(run_n-14) + '.csv'
    elif run_n <= 18:
        run_files = '30_38_2_1.csv'
        # only one file of this denomination
    elif run_n <= 19:
        # only one file of this denomination
        run_files = '41_42_0_1.csv'
    elif run_n <= 24:
        run_files = '41_42_12_' + str(run_n-19) + '.csv'
    elif run_n <= 28:
        run_files = '50_38_6_' + str(run_n-24) + '.csv'
    elif run_n <= 31:
        run_files = '06_42_5_' + str(run_n-28) + '.csv'
    else: 
        print('WARNING: There are no more files in the 2022 dataset to run. Defaulting to the last file.')
        run_files = '06_42_5_4.csv'
    
    # name of the CSV file that will be opened
    csv_file = csv_folder + run_files
    
    # init the number of fruits and the list of fruits for this segment
    n_fruit_file = 0
    x_list_file  = list()
    y_list_file  = list()  # need the y-coordinates to separate fruits into their respective segments
    z_list_file  = list()

    # open the file once and later separate into segments (don't have to open it for every segment)
    with open(csv_file) as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC)
        for row in reader:
            # the 2022 data is the only data set that has different x, y, z definitions than the rest of the project
            y_list_file.append(row[0]) # length along orchard row
            z_list_file.append(row[1]) # height
            x_list_file.append(row[2]) # height
            
            n_fruit_file += 1

    # turn the file's list of z-coordinates into an array for easier work
    x = np.array(x_list_file)
    y = np.array(y_list_file)
    z = np.array(z_list_file)
    
    # remove any negative values and start at a min value for the whole orchard segment 
    x_translate = np.amin(x) - 0.01
    x = x - x_translate
    y_translate = np.amin(y) - 0.01
    y = y - y_translate
    z_translate = np.amin(z) - 0.01
    z = z - z_translate

    sum_length += np.amax(z)
    
    if file_max_x < np.amax(x):
        file_max_x = np.amax(x)
    if file_min_x > np.amin(x):
        file_min_x = np.amin(x)
        
    if file_max_y < np.amax(y):
        file_max_y = np.amax(y)
    if file_min_y > np.amin(y):
        file_min_y = np.amin(y)
        
    if file_max_z < np.amax(z):
        file_max_z = np.amax(z)
    if file_min_z > np.amin(z):
        file_min_z = np.amin(z)
        
    file_N_list.append(len(x))
    
#     for seg in seg_n:
#         # reset y_seg and z_seg to contain all fruits from the CSV (will be cleaned up in a bit)
#         x_file = np.copy(x)
#         y_file = np.copy(y)
#         z_file = np.copy(z)
        
#         y_limit = [seg*y_lim[1], (seg+1)*y_lim[1]] # sets the segments starts and ends at based on the segment number (keep things consistent between runs/datasets)
# #         print('*************** PRINT y_limits ******************', y_limit)
        
#         # only use the fruits within this segment
#         index_out_of_y_bounds = np.where((y <= y_limit[0]) | (y >= y_limit[1]))
#         # remove any fruits that fell outside of the segment
#         x_seg = np.delete(x_seg, index_out_of_y_bounds[0])
#         y_seg = np.delete(y_seg, index_out_of_y_bounds[0])
#         z_seg = np.delete(z_seg, index_out_of_y_bounds[0])
        
#         if len(z_seg) > 0:
#             segment_y_list.append(y_seg)
#             segment_z_list.append(z_seg)

#             segment_N_list.append(len(z_seg))
            
    if len(x) > 300:
        fig, ax = plt.subplots()
        ax.scatter(y, z, color='r')
        ax.set_xlabel('distance along orchard row [m]')
        ax.set_ylabel('distance height [m]')
#                 file_name = './plots/minfpt_cr33_bestv.pdf'
#                 print('Saving plot of FPT and FPE vs density in', file_name)
#                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
        plt.show()
                
            
    if run_n == 3: # index of seg starts at 0
        # min fpt for both partitions, bestV, CR33
        print('     N = %d\n' %len(y_seg))
        
        ax = plt.figure().add_subplot(projection='3d')
        ax.scatter(x, y, z, color='r')
        ax.set_xlabel('into canopy [m]')
        ax.set_ylabel('along orchard row [m]')
        ax.set_zlabel('height [m]')
        fig.tight_layout()
        ax.view_init(elev=12, azim=-166+180)
        file_name = './plots/3d_seminar0.pdf'
        print('Saving plot of FPT and FPE vs density in', file_name)
        plt.savefig(file_name,dpi=300)#, bbox_inches='tight')
        plt.show()

#             elif run_n == 19 and seg == 1:
#                 # max fpt for height partition, bestV, CR33
#                 print('for CR33, bestV the max FPT for height happens in run = 19, seg = 1')
#                 print('     N = %d\n' %len(y_seg))
                
#                 ax = plt.figure().add_subplot(projection='3d')
#                 ax.scatter(x_seg, y_seg, z_seg, color='r')
#                 ax.set_xlabel('distance into canopy [m]')
#                 ax.set_ylabel('distance along orchard row [m]')
#                 ax.set_zlabel('distance height [m]')
#                 ax.view_init(elev=12, azim=-166)
#                 file_name = './plots/maxfpt_height_cr33_bestv.pdf'
#                 print('Saving plot of FPT and FPE vs density in', file_name)
#                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
#                 plt.show()
                
#             elif run_n == 30 and seg == 1:
#                 # max fpt for fruit partition, bestV, CR33
#                 print('for CR33, bestV the min FPT for fruit happens in run = 29, seg = 1')
#                 print('     N = %d\n' %len(y_seg))
                
#                 ax = plt.figure().add_subplot(projection='3d')
#                 ax.scatter(x_seg, y_seg, z_seg, color='r')
#                 ax.set_xlabel('distance into canopy [m]')
#                 ax.set_ylabel('distance along orchard row [m]')
#                 ax.set_zlabel('distance height [m]')
#                 ax.view_init(elev=12, azim=-166)
#                 file_name = './plots/maxfpt_fruit_cr33_bestv.pdf'
#                 print('Saving plot of FPT and FPE vs density in', file_name)
#                 plt.savefig(file_name,dpi=300, bbox_inches='tight')
#                 plt.show()
                
        
# save the number of fruits as an array
N_array    = np.array(file_N_list)
n_segments = len(N_array)

print('there are %d files with fruits in them\n' %n_files)

print('the max values for each axis are:\n        x:              y:              z:')
print('         %3.2f            %3.2f           %3.2f\n' %(file_max_x, file_max_y, file_max_z))
print('the min values for each axis are:\n        x:              y:              z:')
print('         %3.2f            %3.2f           %3.2f\n' %(file_min_x, file_min_y, file_min_z))


print('each segment contains', N_array, 'fruits')
print('\nthere is a total of', sum(N_array), 'fruits over all files')
print('\nover a total length of', sum_length, 'm over all files')


     N = 0



<IPython.core.display.Javascript object>

Saving plot of FPT and FPE vs density in ./plots/3d_seminar0.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

there are 32 files with fruits in them

the max values for each axis are:
        x:              y:              z:
         1.27            14.97           3.47

the min values for each axis are:
        x:              y:              z:
         0.01            0.01           0.01

each segment contains [265 164 158 248 248 187 256 261 339 273 207 194 278 267 168 230 211 158
 126 147 155 308 262 146 204 332 350 276 228 380 281  83] fruits

there is a total of 7390 fruits over all files

over a total length of 97.65730134386973 m over all files


# Analyze the effects of row partitioning by height vs. by fruits

## important parameters

In [17]:
# CSV data
# folder name
# folder = './csv_files/20240130_fcfs_maxV150/'
folder = './csv_files/2024029_partition_speed/'

date   = '2024029'

tested_CRA = np.array(['1/1/1', '1/2/2', '2/1/2', '1/3/3', '3/1/3', '2/2/4', '2/3/6', '3/2/6', '3/3/9'])
tested_CRA_rep = np.array(['1/1/1','1/1/1', '1/2/2','1/2/2', '2/1/2','2/1/2', '1/3/3','1/3/3', '3/1/3','3/1/3', '2/2/4','2/2/4', '2/3/6','2/3/6', '3/2/6','3/2/6', '3/3/9','3/3/9'])
# as ints for looping
file_name_CR = [11, 12, 21, 13, 31, 22, 23, 32, 33]

# x-axis ticks
x = np.arange(len(tested_CRA))

runs_in_file = 111 # check the number of rows in any of the CSVs (-1 because indexes start at 0)

# csv file ending
file_type = '.csv'

# colors
# see https://personal.sron.nl/~pault/#sec:qualitative
# and a different ben (https://stats.stackexchange.com/users/9311/a-different-ben), "Best" series of colors to use for differentiating series in publication-quality plots, URL (version: 2021-03-03): https://stats.stackexchange.com/q/118259
bright_color_array = ['#228833', '#4477AA', '#66CCEE', '#AA3377', '#EE6677', '#CCBB44', '#BBBBBB']
pale_color_array   = ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD']
dark_color_array   = ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555']


## Open CSV results files

In [18]:
# obtain data from CSV to compare hight vs. fruit row partitioning
data_file_start = [date+'_fcfs_2022_data_fruit_CR', date+'_fcfs_2022_data_height_CR']
# data_file_start = [date+'_fcfs_2022_data_fruit_CR', date+'_goal_2022_data_fruit_CR']

############################# V #############################
V_val = 'best'
 
V_run = '_V' + str(V_val)

FPE_all_array = np.zeros([len(tested_CRA)*2, runs_in_file])
FPT_all_array = np.zeros([len(tested_CRA)*2, runs_in_file])
V_all_array   = np.zeros([len(tested_CRA)*2, runs_in_file])
Td_all_array  = np.zeros([len(tested_CRA)*2, runs_in_file])

N_deadspace   = np.zeros([len(tested_CRA)*2, runs_in_file]) # depends on the harvester configuration
N_segment     = np.zeros(runs_in_file)

# Equal number of fruits in each row partition
# arrays to save the FPE and FPT of each C/R/C*R configuration
FPE_array_fruits = np.zeros((len(tested_CRA), runs_in_file))
FPT_array_fruits = np.zeros((len(tested_CRA), runs_in_file))

V_avg_array_fruits = np.zeros(len(tested_CRA))
V_avg_array_height = np.zeros(len(tested_CRA))

# arrays to save the mean and variance of FPE and FPT of each C/R/C*R configuration
FPE_avg_array_fruits = np.zeros(len(tested_CRA))
FPE_std_array_fruits = np.zeros(len(tested_CRA))
FPT_avg_array_fruits = np.zeros(len(tested_CRA))
FPT_std_array_fruits = np.zeros(len(tested_CRA))

# Equal height for each row partition
# arrays to save the FPE and FPT of each C/R/C*R configuration
FPE_array_height = np.zeros((len(tested_CRA), runs_in_file))
FPT_array_height = np.zeros((len(tested_CRA), runs_in_file))

# arrays to save the mean and variance of FPE and FPT of each C/R/C*R configuration
FPE_avg_array_height = np.zeros(len(tested_CRA))
FPE_std_array_height = np.zeros(len(tested_CRA))
FPT_avg_array_height = np.zeros(len(tested_CRA))
FPT_std_array_height = np.zeros(len(tested_CRA))

cr_index = 0
cr_index_all = 0

start = 0

# only look at C/R combinations in the file_name_CR array
for cr in file_name_CR:
    for file_i in range(len(data_file_start)):
        file_name = folder + data_file_start[file_i] + str(cr) + V_run + file_type
        print('\nThe file being opened is', file_name) 

        with open(file_name) as csvfile:

            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            row_i = 0

            for row in reader:
                if cr_index == 0:
                    # save the number of fruits in each array available to the arms (if in dead space, not counted here)
                    N_segment[row_i] = row[3]
                    
                if cr == 11:
                    cr_str = 0
                elif cr == 12:
                    cr_str = 1
                elif cr == 21:
                    cr_str = 2
                elif cr == 13:
                    cr_str = 3
                elif cr == 31:
                    cr_str = 4
                elif cr == 22:
                    cr_str = 5
                elif cr == 23:
                    cr_str = 6
                elif cr == 32:
                    cr_str = 7
                else:
                    cr_str = 8
                    
                new_record = pd.DataFrame([{'C/R/C*R':cr_str, 'V':row[7], 'FPE':row[9], 'FPT':row[11]}])
                
                if file_i == 0:
                    # row by fruit
                    FPE_all_array[cr_index_all,row_i] = row[9]
                    FPT_all_array[cr_index_all,row_i] = row[11]
                    V_all_array[cr_index_all,row_i] = row[7]
#                     Td_all_array[cr_index_all,row_i] = row[13]
                    
                    FPE_array_fruits[cr_index,row_i] = row[9]
                    FPT_array_fruits[cr_index,row_i] = row[11]
#                     print('By fruit: for V %s, row %d' %(V[V_i], row_i))
#                     print('FPE value %3.3f and FPT value %3.3f' %(row[8], row[10]))

                    N_deadspace[cr_index_all,row_i] = row[4] # only need one file type
                    
                    if start == 0:
                        df_fruit = pd.DataFrame([[cr_str, row[7], row[9], row[11]]], columns=['C/R/C*R', 'V', 'FPE', 'FPT'])
                    else:
                        df_fruit = pd.concat([df_fruit, new_record], ignore_index=True)

                elif file_i == 1:
                    # row by height
                    FPE_all_array[cr_index_all+1,row_i] = row[9]
                    FPT_all_array[cr_index_all+1,row_i] = row[11]
                    V_all_array[cr_index_all+1,row_i] = row[7]
#                     Td_all_array[cr_index_all+1,row_i] = row[13]
                                        
                    FPE_array_height[cr_index,row_i] = row[9]
                    FPT_array_height[cr_index,row_i] = row[11]
#                     print('By hieght: for V %d, row %d' %(v, row_i))
#                     print('FPE value %3.3f and FPT value %3.3f' %(row[7], row[9]))
                    N_deadspace[cr_index_all+1,row_i] = row[4] # only need one file type
    
                    if start == 0:
                        df_height = pd.DataFrame([[cr_str, row[7], row[9], row[11]]], columns=['C/R/C*R', 'V', 'FPE', 'FPT'])
                    else:
                        df_height = pd.concat([df_height, new_record], ignore_index=True)

                row_i += 1
        
        # all files for this CR combo have been processed, calculate the average and standard deviation for them
        FPE_avg_array_fruits[cr_index] = np.average(FPE_array_fruits[cr_index,:])
        FPE_std_array_fruits[cr_index] = np.std(FPE_array_fruits[cr_index,:])
        FPT_avg_array_fruits[cr_index] = np.average(FPT_array_fruits[cr_index,:])
        FPT_std_array_fruits[cr_index] = np.std(FPT_array_fruits[cr_index,:])

        FPE_avg_array_height[cr_index] = np.average(FPE_array_height[cr_index,:])
        FPE_std_array_height[cr_index] = np.std(FPE_array_height[cr_index,:])
        FPT_avg_array_height[cr_index] = np.average(FPT_array_height[cr_index,:])
        FPT_std_array_height[cr_index] = np.std(FPT_array_height[cr_index,:])
        
        V_avg_array_fruits[cr_index] = np.average(V_all_array[cr_index_all,:])
        V_avg_array_height[cr_index] = np.average(V_all_array[cr_index_all+1,:])
        
    # If V_val = 'best,' find the run with the highest and lowest FPT for each configuration and each row partition
    if V_val == 'best':
        max_fpt_run_f = np.where(FPT_array_fruits[cr_index,:] == np.amax(FPT_array_fruits[cr_index,:]))
        max_fpt_run_h = np.where(FPT_array_height[cr_index,:] == np.amax(FPT_array_height[cr_index,:]))

        min_fpt_run_f = np.where(FPT_array_fruits[cr_index,:] == np.amin(FPT_array_fruits[cr_index,:]))
        min_fpt_run_h = np.where(FPT_array_height[cr_index,:] == np.amin(FPT_array_height[cr_index,:]))

#         print('\n*Max* FPT in indexes', max_fpt_run_f[0],'for partition by fruit and', max_fpt_run_h[0],'for partition by height')
#         print('            max fpt values:', FPT_array_fruits[cr_index,max_fpt_run_f[0]], 'and', FPT_array_height[cr_index,max_fpt_run_h[0]])
#         print('            chosen V values:', V_all_array[cr_index,max_fpt_run_f[0]], 'and', V_all_array[cr_index,max_fpt_run_h[0]])
#         print('*Min* FPT in indexes', min_fpt_run_f[0],'for partition by fruit and', min_fpt_run_h[0],'for partition by height')
#         print('            min fpt values', FPT_array_fruits[cr_index,min_fpt_run_f[0]], 'and', FPT_array_height[cr_index,min_fpt_run_h[0]])
#         print('            chosen V values:', V_all_array[cr_index,min_fpt_run_f[0]], 'and', V_all_array[cr_index,min_fpt_run_h[0]])
    start = 1
    cr_index += 1
    cr_index_all += 2
    
    
print('\n\nFor equal fruit')
print('------------------------------------------------------')
print('Average FPE values:\n', FPE_avg_array_fruits, '\n')  
print('Standard dev FPE values:\n', FPE_std_array_fruits, '\n')  
print('Average FPT values:\n', FPT_avg_array_fruits, '\n')  
print('Standard dev FPT values:\n', FPT_std_array_fruits)  

print('\n\nFor equal height')
print('------------------------------------------------------')
print('Average FPE values:\n', FPE_avg_array_height, '\n')
print('Standard dev FPE values:\n', FPE_std_array_height, '\n')
print('Average FPT values:\n', FPT_avg_array_height, '\n')
print('Standard dev FPT values:\n', FPT_std_array_height)




The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_fruit_CR11_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_height_CR11_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_fruit_CR12_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_height_CR12_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_fruit_CR21_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_height_CR21_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_fruit_CR13_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_height_CR13_Vbest.csv

The file being opened is ./csv_files/2024029_partition_speed/2024029_fcfs_2022_data_fruit_CR31_Vbest.csv

The file being opened is ./csv_files/2024

In [10]:
# # line plots with mean rolling FPT, V, and N

# plotting_variable = 'FPT'

# x = range(len(file_name_CR))

# mean_colors = [bright_color_array[1], bright_color_array[4]] 

# fig, ax = plt.subplots(1, 1)

# sns.lineplot(ax=ax, data=df_fruit, x="C/R/C*R", y=plotting_variable, color=mean_colors[0])
# sns.lineplot(ax=ax, data=df_height, x="C/R/C*R", y=plotting_variable, color=mean_colors[1])

# ax.legend(handles=ax.lines[:], labels=["FCFS","GP"],bbox_to_anchor=(1., 1.),loc='best', fontsize=9)


# ax.set_xlabel('No. columns/rows/arms')
# ax.set_ylabel('Mean FPE (%)')

# plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8], ['1/1/1', '1/2/2', '2/1/2', '1/3/3', '3/1/3', '2/2/4', '2/3/6', '3/2/6', '3/3/9'])
# # ax.set_xticks(x, file_name_CR, rotation='vertical')
# # ax.set_xticklabels(tested_CRA_rep, rotation=45, fontsize=8)

# fig.tight_layout()

# file_name = './plots/all_'+ V_run + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

## Plot the relationship between chosen V, FPT, and the number of fruits in a segment

### Functions to fit a line to the data

In [11]:
# def exp_func(t, r0, m):    return r0 * np.exp(-m * t)

# def inv_exp_func(N, A, B): return A*(1 - np.exp(-B*N))
# def inv_exp_n_func(N, E, B):     return (E/N) * (1 - np.exp(-B*N))

# # define the linear function
# def linear_func(t, a, b):  return a * t + b

# def log_func(x, A, B):     return A * np.log(x) + B


# def para_func(x, a, b, c): return x*x*a + x*b + c

# def V_func(x, A, B):       return (A * np.log(x) + B) / x

# def fpt_func(n, l):        return l * n  

# def error_metrics(x, pred_x):
#      # calculate and print the error metric values
#     error   = pred_x - x
#     o_error = x - np.mean(x)
#     y_error = pred_x - np.mean(pred_x)
    
#     rmse = np.sqrt(np.mean(np.square(error)))
#     print('  RMSE      = {0:.5f}'.format(rmse))

#     r_sq = 1 - (np.sum(np.square(error)) / np.sum(np.square(o_error)))
#     print('  R-squared = {0:.5f}'.format(r_sq))

#     d = 1 - (np.sum(np.square(error)) / np.sum(np.square(np.abs(y_error) + np.abs(o_error))))
#     print('  d         = {0:.5f}'.format(d), '\n')
    
#     return(d)

In [12]:
# # line fit V vs. N and FPT vs. N
# pred_r_fpt  = list()
# popt_list_fpt = list()
# pred_r_plot_fpt  = list()
# d_error_fpt  = list()

# pred_r_theo  = list()
# popt_list_theo = list()
# pred_r_plot_theo  = list()
# d_error_theo  = list()


# D = 350 # in cm
# FPE = 0.95

# cr_index_all = 0

# N = np.arange(start=1, stop=200, step=5)  # in sec

# # curve fit, popt gives the parameter values
# for i in range(len(file_name_CR)): # there are four arrays of data/four temperatures
#     pred_r_theo.append([])
#     pred_r_fpt.append([])
    
#     # for cleaner plots
#     pred_r_plot_theo.append([])
#     pred_r_plot_fpt.append([])
    
# #     popt, pcov = optimize.curve_fit(V_func, N_segment, V_all_array[cr_index_all,:])
# #     pred_r[i].append(V_func(N_segment, popt[0], popt[1]))
    
#     popt_fpt, pcov_fpt = optimize.curve_fit(inv_exp_func, N_segment, FPT_all_array[cr_index_all,:])
#     pred_r_fpt[i].append(inv_exp_func(N_segment, popt_fpt[0], popt_fpt[1]))
    
#     popt_theo = np.array([popt_fpt[0]*D/FPE, popt_fpt[1]])
    
# #     if i > 0:
# #         popt_theo = np.array([popt_fpt[0]*D/FPE, -popt_fpt[1]*D/FPE])

# #     else:
# #         popt_theo = np.array([popt_fpt[0]*D/FPE, popt_fpt[1]*D/FPE])
    
#     pred_r_theo[i].append(inv_exp_n_func(N_segment, popt_theo[0], popt_theo[1]))
        
    
#     print('   FPT vs. N best fit coefficients are, A = %0.4f (?) and B = %0.4f (?)' %(popt_fpt[0], popt_fpt[1]))
#     d_i_fpt = error_metrics(FPT_all_array[cr_index_all,:], pred_r_fpt[i]) 
    
#     print('   theoretical V vs. N best fit coefficients are, E = %0.4f (?) and B = %0.4f (?)' %(popt_theo[0], popt_theo[1]))
#     d_i_theo = error_metrics(V_all_array[cr_index_all,:], pred_r_theo[i]) 
    
    
#     print('\n')
    
#     # saves the coefficients in a list for later use
#     popt_list_fpt.append(popt_fpt)
    
#     # for plotting 
#     pred_r_plot_fpt[i].append(inv_exp_func(N, popt_fpt[0], popt_fpt[1]))
    
# #     d_error.append(d_i)
#     d_error_fpt.append(d_i_fpt)
    
#     popt_list_theo.append(popt_theo)
#     pred_r_plot_theo[i].append(inv_exp_n_func(N, popt_theo[0], popt_theo[1]))
#     d_error_theo.append(d_i_theo)
    
#     cr_index_all += 2
    
    

In [13]:
# # plot the chosen V against the number of fruits and the FPT against number of fruits
# mean_colors = [bright_color_array[1], bright_color_array[4]] 
# pred_colors = [dark_color_array[0], dark_color_array[4]] 

# # CR = index: 11 = 0 (-), 13 = 6, 23 = 12, 33 = 16

# fruit_i = 16
# height_i = fruit_i+1

# index = int(fruit_i/2)

# fig, axs = plt.subplots(2,1)

# axs[0].grid(True)
# axs[1].grid(True)

# axs[0].scatter(N_segment, V_all_array[fruit_i,:], color=mean_colors[0], label='partition by fruit')
# axs[0].plot(N, pred_r_plot_theo[index][0], color=pred_colors[0], label='predicted')
# axs[0].set_ylabel('best V (cm/s)')

# axs[1].scatter(N_segment, FPT_all_array[fruit_i,:], color=mean_colors[0])
# axs[1].plot(N, pred_r_plot_fpt[index][0], color=pred_colors[0], label='predicted')
# # axs[1].plot(N_segment, ((FPE_all_array[fruit_i,:]/100)*np.log(N_segment) - V_all_array[fruit_i,:]/350), color=mean_colors[1])

# axs[1].set_ylabel('FPT (fruit/s)')


# # axs[0,1].scatter(N_segment, V_all_array[height_i,:], color=mean_colors[1], label='partition by height')
# # axs[1,1].scatter(N_segment, FPT_all_array[height_i,:], color=mean_colors[1])
# # axs[2,1].scatter(N_segment, FPT_all_array[height_i,:], color=mean_colors[1])

# axs[1].set_xlabel('Number of fruits in segment')

# # axs[0].title.set_text('partition by fruit')

# # fig.text(0.70, 0.68-.03, 'Row height equal fruits  ',
# #          backgroundcolor=box_colors[0], color='black', weight='roman',
# #          size='x-small')



# # curve's equation on the plot
# fig.text(.58, .905, r"$ \frac{E*(1-exp(-B*N)}{N} $= $ \frac{%0.3f*(1-exp(-%0.3fN))}{N} $" %(popt_list_theo[index][0],popt_list_theo[index][1]), backgroundcolor='w')
# fig.text(.58, .85, r"$ d $= $ %0.3f $" %(d_error_theo[index]), backgroundcolor='w', size='small')
# # line's equation on the plot
# # fig.text(.75, .42, r"$ m*N $= $ %0.3f * N $" %popt_list_fpt[popt_i][0], backgroundcolor='w', size='small')
# fig.text(.58, .463, r"$ A(1-exp(-BN)) $= $ %0.3f(1-exp(-%0.3fN) $" %(popt_list_fpt[index][0], popt_list_fpt[index][1]), backgroundcolor='w', size='small')
# fig.text(.58, .42, r"$ d $= $ %0.3f $" %(d_error_fpt[index]), backgroundcolor='w', size='small')


# # axs[0,0].set_ylim(0, 20)
# # axs[0,1].set_ylim(0, 20)
# # axs[1,0].set_ylim(0, 105)
# # axs[1,1].set_ylim(0, 105)
# # axs[2,0].set_ylim(0, 2.6)
# # axs[2,1].set_ylim(0, 2.6)
# axs[0].set_ylim(-5, 100)
# axs[1].set_ylim(0, 2.6)
# fig.tight_layout()

# file_name = './plots/N_vs'+ V_run +'_CR' + str(file_name_CR[int(fruit_i/2)]) + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

## Zoomed in look at the lines and curves that show up in scatter plots

### Constant speed, only FPT

In [14]:
# # plot trend lines of line in FPT v. N caused by 100% FPE and constant V
# mean_colors = [bright_color_array[1], bright_color_array[4]] 

# # CR = index: 11 = 0, 13 = 6, 23 = 12, 33 = 16

# fruit_i = 16
# height_i = fruit_i+1

# V_const = 20

# fig, axs = plt.subplots(1, 2)
# axs[0].scatter(N_segment, FPT_all_array[fruit_i,:], color=mean_colors[0])
# axs[0].set_ylabel('FPT (fruit/s)')
# axs[1].scatter(N_segment, FPT_all_array[height_i,:], color=mean_colors[1])

# axs[0].grid(True)
# axs[1].grid(True)

# # if V = constant, slope of line = (V/D) * 1/FPE 
# # for CR33, D = 3.5 + 3*1 + 2*0.15 (might want to add total travel length to the CSV)
# C = 3
# D = (3.5 + C*1 + (C-1)*0.15)*100

# V = np.ones(len(V_all_array[fruit_i,:])) * V_const

# FPE = np.ones(len(FPE_all_array[fruit_i,:])) * 100

# # FPT lines
# # fruit_slope = (V_all_array[fruit_i,:] / (D*100)) * (FPE_all_array[fruit_i,:]/100) * N_segment
# # height_slope = (V_all_array[height_i,:] / (D*100)) * (FPE_all_array[height_i,:]/100) * N_segment
# fruit_slope = (V / D) * (FPE/100) * N_segment
# height_slope = (V / D) * (FPE/100) * N_segment

# axs[0].plot(N_segment, fruit_slope, color='mediumseagreen', linestyle='-', linewidth=1)
# axs[1].plot(N_segment, height_slope, color='mediumseagreen', linestyle='-', linewidth=1, label='V=20 cm/s, FPE=100%')

# axs[0].set_xlabel('Number of fruits in segment')
# axs[1].set_xlabel('Number of fruits in segment')

# axs[0].title.set_text('partition by fruit')
# axs[1].title.set_text('partition by height')

# axs[0].set_ylim(0, 3)
# axs[1].set_ylim(0, 3)

# legend = axs[1].legend(bbox_to_anchor=(1., 1.),loc='best', fontsize=9)

# fig.tight_layout()

# file_name = './plots/N_vs'+ V_run +'_CR' + str(file_name_CR[int(fruit_i/2)]) + '_zoom.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

### Best speed as variable, FPE and FPT

In [15]:
# # plot trend lines of curves and lines in FPE and FPT v. N
# mean_colors = [bright_color_array[1], bright_color_array[4]] 

# # CR = index: 11 = 0, 13 = 6, 23 = 12, 33 = 16

# fruit_i = 6
# height_i = fruit_i+1

# fig, axs = plt.subplots(2, 2)
# axs[0,0].scatter(N_segment, FPE_all_array[fruit_i,:], color=mean_colors[0])
# axs[0,0].set_ylabel('FPE (%)')
# axs[0,1].scatter(N_segment, FPE_all_array[height_i,:], color=mean_colors[1])

# axs[1,0].scatter(N_segment, FPT_all_array[fruit_i,:], color=mean_colors[0])
# axs[1,0].set_ylabel('FPT (fruit/s)')
# axs[1,1].scatter(N_segment, FPT_all_array[height_i,:], color=mean_colors[1])

# axs[0,0].grid(True)
# axs[0,1].grid(True)
# axs[1,0].grid(True)
# axs[1,1].grid(True)

# # if V = constant, slope of line = (V/D) * 1/FPE 
# # for CR33, D = 3.5 + 3*1 + 2*0.15 (might want to add total travel length to the CSV)
# C = 1
# D = (3.5 + C*1 + (C-1)*0.15)*100

# # points = np.arange(0,111)

# V1 = np.ones(len(V_all_array[fruit_i,:])) * 4
# V2 = np.ones(len(V_all_array[fruit_i,:])) * 2

# FPE = np.ones(len(FPE_all_array[fruit_i,:])) * 95
# FPE2 = np.ones(len(FPE_all_array[fruit_i,:])) * 98

#     # FPE curves
# fruit_curve1 = 100* (N_segment - 4) / N_segment 
# height_curve1 = 100* (N_segment - 4) / N_segment 

# fruit_curve2 = 100* (N_segment - 2) / N_segment 
# height_curve2 = 100* (N_segment - 2) / N_segment

#     # FPT lines
# # fruit_slope = (V_all_array[fruit_i,:] / (D*100)) * (FPE_all_array[fruit_i,:]/100) * N_segment
# # height_slope = (V_all_array[height_i,:] / (D*100)) * (FPE_all_array[height_i,:]/100) * N_segment
# fruit_slope1 = (V1 / D) * (FPE/100) * N_segment
# height_slope1 = (V1 / D) * (FPE/100) * N_segment

# fruit_slope2 = (V2 / D) * (FPE/100) * N_segment
# height_slope2 = (V2 / D) * (FPE/100) * N_segment

# fruit_slope3 = (V1 / D) * (FPE2/100) * N_segment
# height_slope3 = (V1 / D) * (FPE2/100) * N_segment

# axs[0,0].scatter(N_segment, fruit_curve2, color='blue', marker='o', s = 3**2)
# axs[0,1].scatter(N_segment, height_curve2, color='blue', marker='o', s = 3**2, label='(N-2)/N')
# axs[0,0].scatter(N_segment, fruit_curve1, color='olivedrab', marker='o', s = 3**2)
# axs[0,1].scatter(N_segment, height_curve1, color='olivedrab', marker='o', s = 3**2, label='(N-4)/N')

# axs[1,0].plot(N_segment, fruit_slope2, color='mediumseagreen', linestyle='-', linewidth=1)
# axs[1,1].plot(N_segment, height_slope2, color='mediumseagreen', linestyle='-', linewidth=1, label='V=2 cm/s, FPE=95%')
# axs[1,0].plot(N_segment, fruit_slope1, color='purple', linestyle='-', linewidth=1)
# axs[1,1].plot(N_segment, height_slope1, color='purple', linestyle='-', linewidth=1, label='V=4 cm/s, FPE=95%')
# axs[1,0].plot(N_segment, fruit_slope3, color='orange', linestyle='-', linewidth=1)
# axs[1,1].plot(N_segment, height_slope3, color='orange', linestyle='-', linewidth=1, label='V=4 cm/s, FPE=98%')


# axs[1,0].set_xlabel('Number of fruits in segment')
# axs[1,1].set_xlabel('Number of fruits in segment')

# axs[0,0].title.set_text('partition by fruit')
# axs[0,1].title.set_text('partition by height')

# axs[0,0].set_ylim(90, 102)
# axs[0,1].set_ylim(90, 102)
# axs[1,0].set_ylim(0, 1.5)
# axs[1,1].set_ylim(0, 1.5)

# legend = axs[0,1].legend(bbox_to_anchor=(1., 1.),loc='best', fontsize=9)
# legend = axs[1,1].legend(bbox_to_anchor=(1., 1.),loc='best', fontsize=9)

# fig.tight_layout()

# file_name = './plots/N_vs'+ V_run +'_CR' + str(file_name_CR[int(fruit_i/2)]) + '_zoom.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

## Create Latex table with the mean of the FPE and FPT for both partitioning methods
(Use only for "best" speed because a later table aggregates all the other speeds together)

In [16]:
# table_title = ['FPE', 'FPT']
# table_units = ['\%', 'fruits/s']
# table_thing = ['mean', 'variance']
# table_test  = ['fruits', 'height']

# column_size = 8 

# for title_i in range(len(table_title)):
    
# #     if test_i < 1:
# #         # FPE has longer results which makes the tables longer. This resizes the tables so they are about the 
# #         # right size compared with each other.
# #         column_size = 8 
# #     else:
# #         column_size = 7
    
#     print('\\begin{table}[h]')
#     print('\\centering')
#     print('\\resizebox{0.%d\columnwidth}{!}{\\begin{tabular}{|c | c c c c c c c c c|}' %column_size)

# #     # for both FPE and FPT 
#     print('\\multicolumn{10}{c}{mean %s for both partition methods} \\\\' %table_title[title_i])
#     print('\\hline')
#     print('\\multicolumn{1}{|c |}{ partition method } & \\multicolumn{9}{c|}{Harvester configuration} \\\\')
#     print('\\multicolumn{1}{|c |}{ } & \\multicolumn{9}{c|}{ (C/R/C*R)} \\\\')
#     print('\\hline')
#     print(' & 1/1/1 & 1/2/2 & 2/1/2 & 1/3/3 & 3/1/3 & 2/2/4 & 2/3/6 & 3/2/6 & 3/3/9 \\\\')
#     print('\\hline')
    
#     for test_i in range(len(table_test)):
#         for thing_i in range(len(table_thing)):
        
    
#             if title_i == 0:
#                 # FPE
#                 if thing_i == 0:
#                     # mean
#                     if test_i == 0:
#                         # partition by fruit
#                         array = np.copy(FPE_avg_array_fruits)
#                     else:
#                         # partition by height
#                         array = np.copy(FPE_avg_array_height)
#                 else: 
#                     # variance
#                     if test_i == 0:
#                         # partition by fruit
#                         array = np.copy(FPE_std_array_fruits)
#                     else:
#                         # partition by height
#                         array = np.copy(FPE_std_array_height)
#             else:
#                 # FPT
#                 if thing_i == 0:
#                     # mean
#                     if test_i == 0:
#                         # partition by fruit
#                         array = np.copy(FPT_avg_array_fruits)
#                     else:
#                         # partition by height
#                         array = np.copy(FPT_avg_array_height)
#                 else: 
#                     # variance
#                     if test_i == 0:
#                         # partition by fruit
#                         array = np.copy(FPT_std_array_fruits)
#                     else:
#                         # partition by height
#                         array = np.copy(FPT_std_array_height)

#             print(f'%s (%s) & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f \\\\' %(table_test[test_i], table_thing[thing_i], array[0], array[1], array[2], array[3], array[4], array[5], array[6], array[7], array[8]))
        

            
# #         print('\\hline')
# #         print('\\multicolumn{10}{c}{ } \\\\')

#     # finish the tabular format
#     print('\\hline')
#     print('\\end{tabular}}')
#     print('\\caption{%s mean and variance for all $111$ orchard row segments, all nine harvester configurations, and all six speeds. Scheduling done using FCFS and row partition by %s.}' %(table_title[test_i], partition_by))
#     print('\\label{table:Vbest_mean_std}')
#     print('\\end{table}')
#     print('%')


## Obtain the average number of fruits in deadspace of each harvester configuration

In [17]:
# ## Look at how the configuration affects the number of fruits due to dead space
# print('shape of N_available_array', N_deadspace.shape)
# print('shape of N_array', N_segment.shape)

# # print('Total N in each segment\n', N_segment)
# # print('\n')
# print('Available N in each segment\n', N_deadspace)
# # print('\n')

# n_subplots = 1

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# # Now fill the boxes with desired colors
# box_colors = [pale_color_array[1], pale_color_array[4]]
# mean_colors = [bright_color_array[1], bright_color_array[4]] 
# num_boxes = len(tested_CRA_rep)
# medians = np.empty(num_boxes)

# # if isinstance(V_val, int):
# #     fig.suptitle(f'V=%d cm/s, comparison of row partitioning by fruit vs. by height \n over nine configurations' %V_val)
# # else:
# #     fig.suptitle(f'V=%s cm/s, comparison of row partitioning by fruit vs. by height \n over nine configurations' %V_val)
    
# bp = axs.boxplot(N_deadspace.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between Dark Khaki and Royal Blue
#     axs.add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs.plot(median_x, median_y, 'r', linewidth=2)
#     medians[i] = median_y[0]

# axs.set_xticklabels(tested_CRA_rep,
#                     rotation=45, fontsize=8)    

# axs.set_xlim(0.5, num_boxes + 0.5)
# axs.set_ylim(np.amin(N_deadspace) - 0.05, np.amax(N_deadspace) + 0.3)

# axs.set_xticklabels(tested_CRA_rep)
# axs.set_ylabel('Number of fruits in segment deadspace')
# axs.set_xlabel('No. columns/rows/arms')

# # Finally, add a basic legend
# fig.text(0.70, 0.68-.03, 'Row height equal fruits  ',
#          backgroundcolor=box_colors[0], color='black', weight='roman',
#          size='x-small')
# fig.text(0.70, 0.645-.03, 'Row height equal height',
#          backgroundcolor=box_colors[1], color='black', weight='roman', 
#          size='x-small')

# fig.tight_layout()

# file_name = './plots/boxplot_deadspace.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300)

# plt.show()

## Obtain a boxplot of the handling time of each configuration

In [18]:
# ## Look at how the configuration affects the number of fruits due to dead space

# n_subplots = 1

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# # Now fill the boxes with desired colors
# box_colors = [pale_color_array[1], pale_color_array[4]]
# mean_colors = [bright_color_array[1], bright_color_array[4]] 
# num_boxes = len(tested_CRA_rep)
# medians = np.empty(num_boxes)

# # if isinstance(V_val, int):
# #     fig.suptitle(f'V=%d cm/s, comparison of row partitioning by fruit vs. by height \n over nine configurations' %V_val)
# # else:
# #     fig.suptitle(f'V=%s cm/s, comparison of row partitioning by fruit vs. by height \n over nine configurations' %V_val)
    
# bp = axs.boxplot(Td_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between Dark Khaki and Royal Blue
#     axs.add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs.plot(median_x, median_y, 'r', linewidth=2)
#     medians[i] = median_y[0]

# axs.set_xticklabels(tested_CRA_rep,
#                     rotation=45, fontsize=8)    

# axs.set_xlim(0.5, num_boxes + 0.5)
# axs.set_ylim(np.amin(N_deadspace) - 0.05, np.amax(N_deadspace) + 0.3)

# axs.set_xticklabels(tested_CRA_rep)
# axs.set_ylabel('Mean handling time [s/fruit]')
# axs.set_xlabel('No. columns/rows/arms')

# # Finally, add a basic legend
# fig.text(0.70, 0.68-.03, 'equal fruits',
#          backgroundcolor=box_colors[0], color='black', weight='roman',
#          size='x-small')
# fig.text(0.70, 0.645-.03, 'equal height',
#          backgroundcolor=box_colors[1], color='black', weight='roman', 
#          size='x-small')

# # file_name = './plots/boxplot_deadspace.pdf'
# # print('Saving plot of FPT and FPE vs density in', file_name)
# # plt.savefig(file_name,dpi=300)

# plt.show()

## Compare height vs fruit partitioning for one V using a boxplot

In [47]:
# #### Plot boxplot comparison of row partitioning height vs. fruits

# if isinstance(V_val, int):
#     n_subplots = 2
# else:
#     n_subplots = 3

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# # Now fill the boxes with desired colors
# box_colors = [pale_color_array[1], pale_color_array[4]]
# mean_colors = [bright_color_array[1], bright_color_array[4]] 
# num_boxes = len(tested_CRA_rep)
# medians = np.empty(num_boxes)

# # if isinstance(V_val, int):
# #     fig.suptitle(f'V=%d cm/s, comparison of row partitioning by fruit vs. by height \n over nine configurations' %V_val)
# # else:
# #     fig.suptitle(f'V=%s cm/s, comparison of row partitioning by fruit vs. by height \n over nine configurations' %V_val)

# ############################## FPE ##################################
# bp = axs[0].boxplot(FPE_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between Dark Khaki and Royal Blue
#     axs[0].add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs[0].plot(median_x, median_y, 'r', linewidth=2)
#     medians[i] = median_y[0]
#     # Finally, overplot the sample averages, with horizontal alignment
#     # in the center of each box
# #     axs[0].plot(np.average(med.get_xdata()), np.average(FPE_all_array.T[i]),
# #              color=mean_colors[i % 2], marker='o', markeredgecolor='k')

# y_95_line = np.ones(len(tested_CRA_rep))*95
# x_95_line = np.arange(1, len(tested_CRA_rep)+1)
# print(x_95_line)
# axs[0].plot(x_95_line , y_95_line, color='r', marker=' ', linestyle='--', linewidth=1, label='95% FPE threshold')

# axs[0].set_xlim(0.5, num_boxes + 0.5)
# axs[0].set_xticklabels([ ])
# axs[0].set_ylabel('FPE (%)')

# ############################## FPT ##################################
# bp = axs[1].boxplot(FPT_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between colors depending on data
#     axs[1].add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs[1].plot(median_x, median_y, color='r', linewidth=2)
#     medians[i] = median_y[0]
#     # Finally, overplot the sample averages, with horizontal alignment
#     # in the center of each box
# #     axs[1].plot(np.average(med.get_xdata()), np.average(FPT_all_array.T[i]),
# #              color=mean_colors[i % 2], marker='o', markeredgecolor='k')

# axs[1].set_ylabel('FPT (fruits/s)')


# if not isinstance(V_val, int):
#     ############################## V ##################################
#     bp = axs[2].boxplot(V_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
#     plt.setp(bp['boxes'], color='black')
#     plt.setp(bp['whiskers'], color='black')
#     # plt.setp(bp['fliers'], color='red', marker='+')

#     for i in range(num_boxes):
#         box = bp['boxes'][i]
#         box_x = []
#         box_y = []
#         for j in range(5):
#             box_x.append(box.get_xdata()[j])
#             box_y.append(box.get_ydata()[j])
#         box_coords = np.column_stack([box_x, box_y])
#         # Alternate between colors depending on data
#         axs[2].add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#         # Now draw the median lines back over what we just filled in
#         med = bp['medians'][i]
#         median_x = []
#         median_y = []
#         for j in range(2):
#             median_x.append(med.get_xdata()[j])
#             median_y.append(med.get_ydata()[j])
#             axs[2].plot(median_x, median_y, color='r', linewidth=2)
#         medians[i] = median_y[0]
#         # Finally, overplot the sample averages, with horizontal alignment
#         # in the center of each box
#     #     axs[1].plot(np.average(med.get_xdata()), np.average(FPT_all_array.T[i]),
#     #              color=mean_colors[i % 2], marker='o', markeredgecolor='k')

#     axs[2].set_ylabel('best V (cm/s)')
    
#     # remove y-ticks from FPT
#     axs[1].set_xticklabels([ ])

# axs[-1].set_xlabel('No. columns/rows/arms')
# axs[-1].set_xlim(0.5, num_boxes + 0.5)
# axs[-1].set_xticklabels(tested_CRA_rep,
#                     rotation=45, fontsize=8)

# fig.tight_layout()

# if not isinstance(V_val, int):
#     # Finally, add a basic legend
#     fig.text(0.15, 0.68-.05, 'equal fruits  ',
#              backgroundcolor=box_colors[0], color='black', weight='roman',
#              size='x-small')
#     fig.text(0.15, 0.645-.05, 'equal height',
#              backgroundcolor=box_colors[1], color='black', weight='roman', 
#              size='x-small')

# elif V_val < 10:
#     # Finally, add a basic legend
#     fig.text(0.70, 0.68-.03, 'equal fruits  ',
#              backgroundcolor=box_colors[0], color='black', weight='roman',
#              size='x-small')
#     fig.text(0.70, 0.645-.03, 'equal height',
#              backgroundcolor=box_colors[1], color='black', weight='roman', 
#              size='x-small')
#     # fig.text(0.70, 0.615-.03, marker='o', color='black', weight='roman', size='medium')
#     # fig.text(0.715, 0.613-.03, ' Average value', color='black', weight='roman',
#     #          size='x-small')
    
# else:
#     # Finally, add a basic legend
#     fig.text(0.20, 0.48-.03, 'equal fruits  ',
#              backgroundcolor=box_colors[0], color='black', weight='roman',
#              size='x-small')
#     fig.text(0.20, 0.445-.03, 'equal height',
#              backgroundcolor=box_colors[1], color='black', weight='roman', 
#              size='x-small')
    
# file_name = './plots/boxplot_row_part' + V_run + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300)
    
# plt.show()

<IPython.core.display.Javascript object>

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Saving plot of FPT and FPE vs density in ./plots/boxplot_row_part_Vbest.pdf


## Plot the average FPE and FPT in a line plot

In [36]:
# #### Plot average FPE and FPT of height

# n_subplots = 3

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# y_95_line = np.ones(len(x))*95

# # fig, axs = plt.subplots(n_subplots, 1)
# # ## want subplots to stack in columns so they can be compared, see
# # ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# axs[0].plot(x , y_95_line, color='r', marker=' ', linestyle='--', linewidth=1, label='95% FPE threshold')

# axs[0].plot(x, FPE_avg_array_fruits, color=bright_color_array[1], marker='s', markersize=5, linewidth=2, label='equal fruits')
# axs[0].plot(x, FPE_avg_array_height, color=bright_color_array[4], marker='s', markersize=5, linewidth=2, label='equal height')
# # axs[0].errorbar(x , FPE_avg_array_height, yerr=FPE_std_array_height, capsize=5, color=color_array[4])
# # axs[0].errorbar(x , FPE_avg_array_fruits, yerr=FPE_std_array_fruits, capsize=3, color=color_array[1])
# ## Labels and grid for ax0
# # axs[0].set_xlabel('No. Rows x No. Columns')
# axs[0].set_ylabel('mean FPE (%)')
# axs[0].grid(True)
# axs[0].set_xticklabels([ ])

# # axs[0].set_title('Results from running a min makespan MIP model \n to pick a vehicle velocity that results in the best FPT \n when FPE is at least 95%')
# # axs[0].set_title(' ')
# legend = axs[0].legend(bbox_to_anchor=(1., 1.),loc='best', fontsize=9)

# axs[1].plot(x, FPT_avg_array_fruits, color=bright_color_array[1], marker='s', markersize=5, linewidth=2, label='equal fruits')
# axs[1].plot(x, FPT_avg_array_height, color=bright_color_array[4], marker='s', markersize=5, linewidth=2, label='equal height')
# # axs[1].errorbar(x , FPT_avg_array_height, yerr=FPT_std_array_height, capsize=5, color=color_array[4])
# # axs[1].errorbar(x , FPT_avg_array_fruits, yerr=FPT_std_array_fruits, capsize=3, color=color_array[1])
# # axs[1].set_xlabel('Columns/Rows/Total Arms')
# axs[1].grid(True)
# axs[1].set_ylabel('mean FPT (f/s)')
# axs[1].set_xticklabels([ ])

# axs[2].plot(x, V_avg_array_fruits, color=bright_color_array[1], marker='s', markersize=5, linewidth=2, label='equal fruits')
# axs[2].plot(x, V_avg_array_height, color=bright_color_array[4], marker='s', markersize=5, linewidth=2, label='equal height')
# # axs[1].errorbar(x , FPT_avg_array_height, yerr=FPT_std_array_height, capsize=5, color=color_array[4])
# # axs[1].errorbar(x , FPT_avg_array_fruits, yerr=FPT_std_array_fruits, capsize=3, color=color_array[1])
# # axs[1].set_xlabel('Columns/Rows/Total Arms')
# axs[2].grid(True)
# axs[2].set_ylabel('mean best V (cm/s)')

# # plt.xticks(x, tested_CRA)#, rotation='vertical')
# axs[-1].set_xticks(x, tested_CRA, rotation=45)
# axs[-1].set_xlabel('No. columns/rows/arms')

# axs[0].set_ylim(90, 101)
# axs[1].set_ylim(0, 2.3)
# axs[2].set_ylim(0, 31)

# fig.tight_layout()

# file_name = './plots/mean'+ V_run + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')
# plt.show()

<IPython.core.display.Javascript object>

Saving plot of FPT and FPE vs density in ./plots/mean_Vbest.pdf


## Perform t-tests to determine if there is a difference between partitioning methods

In [21]:
# # for t-tests printed out as latex table

# # latex table heading
# # print as latex table

# print('\\begin{table}')
# print('\\centering')
# print('\\begin{tabular}{|c |c c |c c|}')
# print('\\multicolumn{5}{c}{ V =', V_val, 'cm/s} \\\\')
# print('\\hline')
# print('\\multicolumn{1}{|c|}{ } & \\multicolumn{2}{|c|}{FPE} & \\multicolumn{2}{|c|}{FPT} \\\\')
# print('C/R/C*R & p-value & Decision & p-value & Decision \\\\')
# print('\\hline')

# # explanation: https://www.jmp.com/en_us/statistics-knowledge-portal/t-test/two-sample-t-test.html
# # scipy.stats: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# for cr_index in range(len(tested_CRA)):
#     # perform an individual t-test for FPE and FPE for every C/R/C*R combo
#     fpe_t = stats.ttest_ind(FPE_array_height[cr_index,:], FPE_array_fruits[cr_index,:], equal_var=False, nan_policy='omit')
#     fpt_t = stats.ttest_ind(FPT_array_fruits[cr_index,:], FPT_array_height[cr_index,:], equal_var=False, nan_policy='omit')
    
#     if fpe_t.pvalue < 0.05:
# #         print('         ** Reject the null, there is a difference in the mean FPE **')
#         reject_fpe = '\textbf{Reject the null}'
#     else:
#         reject_fpe = 'Accept the null'
        
#     if fpt_t.pvalue < 0.05:
# #         print('         ** Reject the null, there is a difference in the mean FPT **')
#         reject_fpt = '\textbf{Reject the null}'
#     else:
#         reject_fpt = 'Accept the null'
           
#     print(f'%s & %2.2f & %s & %2.2f & %s \\\\' %(tested_CRA[cr_index],fpe_t.pvalue, reject_fpe, fpt_t.pvalue, reject_fpt))
    
# # finish the tabular format
# print('\\hline')
# print('\\end{tabular}')
# print('\\caption{Results of the t-test comparing the FPE and FPT means of row partitioning by fruit versus by height for all configurations at V =', V_val, 'cm/s.}')
# print('\\label{table:ttest_row_v%s}' %V_val)
# print('\\end{table}')



# Compare results for all V= 5, 10, 20, best cm/s if partitioning by fruit

## Important parameters

In [22]:
# # CSV data
# # folder name
# folder = './csv_files/20240127_FCFS/'
# # folder = './csv_files/'

# date   = '20240127'

# tested_CRA = np.array(['1/1/1', '1/2/2', '2/1/2', '1/3/3', '3/1/3', '2/2/4', '2/3/6', '3/2/6', '3/3/9'])
# tested_CRA_rep = np.array(['1/1/1','1/1/1', '1/2/2','1/2/2', '2/1/2','2/1/2', '1/3/3','1/3/3', '3/1/3','3/1/3', '2/2/4','2/2/4', '2/3/6','2/3/6', '3/2/6','3/2/6', '3/3/9','3/3/9'])
# # as ints for looping
# file_name_CR = [11, 12, 21, 13, 31, 22, 23, 32, 33]

# V = ['1', '5', '10', '15', '20', '100', 'best']
# partition_by = 'fruit'

# data_file_start = date+'_2022_data_'+partition_by+'_CR'

# # x-axis ticks
# x = np.arange(len(tested_CRA))

# runs_in_file = 111 # check the number of rows in any of the CSVs (-1 because indexes start at 0)

# # csv file ending
# file_type = '.csv'

# # colors
# # see https://personal.sron.nl/~pault/#sec:qualitative
# # and a different ben (https://stats.stackexchange.com/users/9311/a-different-ben), "Best" series of colors to use for differentiating series in publication-quality plots, URL (version: 2021-03-03): https://stats.stackexchange.com/q/118259
# bright_color_array = ['#228833', '#4477AA', '#66CCEE', '#AA3377', '#EE6677', '#CCBB44', '#BBBBBB']
# pale_color_array   = ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD']
# dark_color_array   = ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555']


## Open CSV file 

In [23]:
# # # read a CSV file comparing V FPE and FPT for each C/R/C*R setting

# print('Partitioning rows by %s' %partition_by)

# # number of fruits equal
# FPE_array_fruits = np.zeros((len(tested_CRA), runs_in_file, len(V)))
# FPT_array_fruits = np.zeros((len(tested_CRA), runs_in_file, len(V)))

# FPE_avg_array_fruits = np.zeros((len(tested_CRA), len(V)))
# FPE_std_array_fruits = np.zeros((len(tested_CRA), len(V)))
# FPT_avg_array_fruits = np.zeros((len(tested_CRA), len(V)))
# FPT_std_array_fruits = np.zeros((len(tested_CRA), len(V)))

# cr_index = 0

# # only column increase combinations
# for cr in file_name_CR:
#     for V_i in range(len(V)):
#         file_name = folder + data_file_start + str(cr) + '_V' + V[V_i] + file_type
#         print('\nThe file being opened is', file_name) 

#         with open(file_name) as csvfile:

#             reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
#             row_i = 0

#             for row in reader:
#                 FPE_array_fruits[cr_index,row_i, V_i] = row[9]
#                 FPT_array_fruits[cr_index,row_i, V_i] = row[11]
# #                 print('By fruit: for V %s, row %d' %(V[V_i], row_i))
# #                 print('FPE value %3.3f and FPT value %3.3f' %(row[8], row[10]))

#                 row_i += 1

#         # all files for this V have been processed, calculate the average and standard deviation for them
#         FPE_avg_array_fruits[cr_index,V_i] = np.average(FPE_array_fruits[cr_index,:,V_i])
#         FPE_std_array_fruits[cr_index,V_i] = np.std(FPE_array_fruits[cr_index,:,V_i])
#         FPT_avg_array_fruits[cr_index,V_i] = np.average(FPT_array_fruits[cr_index,:,V_i])
#         FPT_std_array_fruits[cr_index,V_i] = np.std(FPT_array_fruits[cr_index,:,V_i])

#     cr_index += 1

    
# print('\n\nFor equal fruit')
# print('------------------------------------------------------')
# print('Average FPE values:\n', FPE_avg_array_fruits, '\n')  
# print('Standard dev FPE values:\n', FPE_std_array_fruits, '\n')  
# print('Average FPT values:\n', FPT_avg_array_fruits, '\n')  
# print('Standard dev FPT values:\n', FPT_std_array_fruits)  


## Build table for FPE and FPT mean and variance for all runs 

In [24]:
# table_title = ['FPE', 'FPT']
# table_units = ['\%', 'fruits/s']
# table_thing = ['mean', 'variance']

# for test_i in range(len(table_title)):
    
#     if test_i < 1:
#         # FPE has longer results which makes the tables longer. This resizes the tables so they are about the 
#         # right size compared with each other.
#         column_size = 8 
#     else:
#         column_size = 7
    
#     print('\\begin{table}[h]')
#     print('\\centering')
#     print('\\resizebox{0.%d\columnwidth}{!}{\\begin{tabular}{|c | c c c c c c c c c|}' %column_size)

#     for thing_i in range(len(table_thing)):
#     # for both FPE and FPT 
#         print('\\multicolumn{10}{c}{%s %s when partition by %s} \\\\' %(table_title[test_i], table_thing[thing_i], partition_by))
#         print('\\hline')
#         print('\\multicolumn{1}{|c |}{ V } & \\multicolumn{9}{c|}{Harvester configuration} \\\\')
#         print('\\multicolumn{1}{|c |}{ (cm/s) } & \\multicolumn{9}{c|}{ (C/R/C*R)} \\\\')
#         print('\\hline')
#         print(' & 1/1/1 & 1/2/2 & 2/1/2 & 1/3/3 & 3/1/3 & 2/2/4 & 2/3/6 & 3/2/6 & 3/3/9 \\\\')
#     #     print('C/R/C*R & p-value & Decision & p-value & Decision \\\\')
#         print('\\hline')

#         for v_i in range(len(V)): # remove best for now
#             # all mean values
#             if test_i == 0:
#                 # FPE
#                 if thing_i == 0:
#                     # mean
#                     array = np.copy(FPE_avg_array_fruits)
#                 else:
#                     array = np.copy(FPE_std_array_fruits)
#             else:
#                 # FPT
#                 if thing_i == 0:
#                     # mean
#                     array = np.copy(FPT_avg_array_fruits)
#                 else:
#                     array = np.copy(FPT_std_array_fruits)

#             print(f'%s & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f & %3.3f \\\\' %(V[v_i], array[0,v_i], array[1,v_i], array[2,v_i], array[3,v_i], array[4,v_i], array[5,v_i], array[6,v_i], array[7,v_i], array[8,v_i]))

#         print('\\hline')
#         print('\\multicolumn{10}{c}{ } \\\\')

#     # finish the tabular format
#     print('\\hline')
#     print('\\end{tabular}}')
#     print('\\caption{%s mean and variance for all $111$ orchard row segments, all nine harvester configurations, and all six speeds. Scheduling done using FCFS and row partition by %s.}' %(table_title[test_i], partition_by))
#     print('\\label{table:%s_%s_%s}' %(table_title[test_i], table_thing[thing_i], partition_by))
#     print('\\end{table}')
#     print('%')


## Line plot of FPE and FPT for all harvester speeds and all nine harvester configurations

In [25]:
# # ## Line subplots

# n_subplots = 2

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py
# for V_i in range(len(V)):
#     if V_i < 6:
#         line_style = '--'
#         line_width = 1
#         cap_size   = 3
#         line_color = bright_color_array[V_i]
#     else:
#         line_style = '-'
#         line_width = 2
#         cap_size   = 5
#         line_color = 'k'
        
#     axs[0].plot(x , FPE_avg_array_fruits[:,V_i], color=line_color, marker='s', markersize=5, linestyle=line_style, linewidth=line_width, label='V = ' + V[V_i])
# #     axs[0].errorbar(x , FPE_avg_array_fruits[:,V_i], yerr=FPE_std_array_fruits[:,V_i], color=bright_color_array[V_i], linestyle=line_style, linewidth=line_width, capsize=cap_size)
    
# ## Labels and grid for ax0
# # axs[0].set_xlabel('No. Rows x No. Columns')
# axs[0].set_ylabel('mean FPE (%)')
# axs[0].grid(True)

# # axs[0].set_title('Results from running a min makespan MIP model \n to pick a vehicle velocity that results in the best FPT \n when FPE is at least 95%')
# # axs[0].set_title(' ')
# legend = axs[0].legend(bbox_to_anchor=(1., 1.3),loc='best', fontsize=15)

# for V_i in range(len(V)):
#     if V_i < 6:
#         line_style = '--'
#         line_width = 1
#         cap_size   = 3
#         line_color = bright_color_array[V_i]
#     else:
#         line_style = '-'
#         line_width = 2
#         cap_size   = 5
#         line_color = 'k'
        
#     axs[1].plot(x, FPT_avg_array_fruits[:,V_i], color=line_color, marker='s', markersize=5, linestyle=line_style, linewidth=line_width)
# #     axs[1].errorbar(x, FPT_avg_array_fruits[:,V_i], yerr=FPT_std_array_fruits[:,V_i], color=bright_color_array[V_i],linestyle=line_style, linewidth=line_width, capsize=cap_size)
# # axs[1].set_xlabel('Columns/Rows/Total Arms')
# axs[1].grid(True)
# axs[1].set_ylabel('mean FPT (fruits/s)')
# # legend = axs[1].legend(bbox_to_anchor=(1., 1),loc='best', fontsize=15)

# axs[n_subplots-1].set_xlabel('No. columns/rows/arms')

# # plt.xticks(x, tested_CRA)#, rotation='vertical')
# axs[0].set_xticks(x, tested_CRA, rotation=45)
# axs[1].set_xticks(x, tested_CRA, rotation=45)

# fig.tight_layout()

# file_name = './plots/all_v_bestV.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300)
# plt.show()

# Compare scheduling algorithms

## Important parameters

In [14]:
# CSV data
# folder name
folder = './csv_files/20240211_algo_compare/'

date   = '20240211'

tested_CRA = np.array(['1/1/1', '1/3/3', '2/3/6', '3/3/9'])#'1/1/1', '1/2/2', '2/1/2', '1/3/3', '3/1/3', '2/2/4', '2/3/6', '3/2/6', '3/3/9'])
tested_CRA_rep = np.array(['1/1/1','1/1/1', '1/3/3', '1/3/3', '2/3/6', '2/3/6','3/3/9', '3/3/9'])#'1/1/1','1/1/1', '1/2/2','1/2/2', '2/1/2','2/1/2', '1/3/3','1/3/3', '3/1/3','3/1/3', '2/2/4','2/2/4', '2/3/6','2/3/6', '3/2/6','3/2/6', '3/3/9','3/3/9'])
# as ints for looping
# file_name_CR = [11, 12, 21, 13, 31, 22, 23, 32, 33]
file_name_CR = [11,13,23,33]

# sched_algo = ['goal', 'epsilon']
sched_algo = ['fcfs', 'goal']#, 'epsilon']

# x-axis ticks
x = np.arange(len(tested_CRA))

runs_in_file = 35#157 # check the number of rows in any of the CSVs (-1 because indexes start at 0)

# csv file ending
file_type = '.csv'

# colors
# see https://personal.sron.nl/~pault/#sec:qualitative
# and a different ben (https://stats.stackexchange.com/users/9311/a-different-ben), "Best" series of colors to use for differentiating series in publication-quality plots, URL (version: 2021-03-03): https://stats.stackexchange.com/q/118259
bright_color_array = ['#228833', '#4477AA', '#66CCEE', '#AA3377', '#EE6677', '#CCBB44', '#BBBBBB']
pale_color_array   = ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD']
dark_color_array   = ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555']


## Open CSV files

In [15]:
# all runs use partition by fruits and attempt to find the best V (range 1-20 cm/s)

FPE_all_array = np.zeros([len(tested_CRA)*2, runs_in_file])
FPT_all_array = np.zeros([len(tested_CRA)*2, runs_in_file])
V_all_array   = np.zeros([len(tested_CRA)*2, runs_in_file])
# Td_all_array  = np.zeros([len(tested_CRA)*2, runs_in_file])

N_segment     = np.zeros(runs_in_file)

# line plots with mean rolling FPT, V, and N

cr_index = 0
cr_index_all = 0

start = 0

# only look at C/R combinations in the file_name_CR array
for cr in file_name_CR:
    for algo_i in range(len(sched_algo)):
        file_name = folder + date+'_'+sched_algo[algo_i]+'_2022_data_fruit_CR' + str(cr) + '_Vbest' + file_type
        print('\nThe file being opened is', file_name) 
        
        with open(file_name) as csvfile:

            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            row_i = 0

            for row in reader:
                if row_i < runs_in_file:
                    if cr_index == 0:
                        # save the number of fruits in each array available to the arms (if in dead space, not counted here)
                        N_segment[row_i] = row[3]
                    
                    if cr == 11:
                        cr_str = 0
                    elif cr == 13:
                        cr_str = 1
                    elif cr == 23:
                        cr_str = 2
                    else:
                        cr_str = 3
#                     cr_str = int(str(int(row[5])) + str(int(row[6]))) 
                    
                    # new_record = pd.DataFrame([{'C/R/C*R':cr_str, 'V':row[7], 'FPE':row[9], 'FPT':row[11]}])

                    if algo_i == 0:
                        # FCFS
                        FPE_all_array[cr_index_all,row_i] = row[9]
                        FPT_all_array[cr_index_all,row_i] = row[11]
                        V_all_array[cr_index_all,row_i] = row[7]
                        
                        # if start == 0:
                        #     df_fcfs = pd.DataFrame([[cr_str, row[7], row[9], row[11]]], columns=['C/R/C*R', 'V', 'FPE', 'FPT'])
                        # else:
                        #     df_fcfs = pd.concat([df_fcfs, new_record], ignore_index=True)
                            
                    elif algo_i == 1:
                        # goal prog
                        FPE_all_array[cr_index_all+1,row_i] = row[9]
                        FPT_all_array[cr_index_all+1,row_i] = row[11]
                        V_all_array[cr_index_all+1,row_i] = row[7]
                        
                        # if start == 0:
                        #     df_goal = pd.DataFrame([[cr_str, row[7], row[9], row[11]]], columns=['C/R/C*R', 'V', 'FPE', 'FPT'])
                        # else:
                        #     df_goal = pd.concat([df_goal, new_record], ignore_index=True)

    
                row_i += 1
                start = 1

    cr_index += 1
    cr_index_all += 2
    
    
# Td = 2.75
# print('N\n', N_segment)
# print('N*PCT_avg\n', N_segment*Td)
# print()
# print('V_1arm = D/(N*PCT_avg)\n', 350/(N_segment*Td))
# print('V_9arm = 9*D/(N*PCT_avg)\n', 9*350/(N_segment*Td))
# print()
# print('theoretical FPT_0.95,1arm = C*R/PCT\n', 1/(Td*.95))
# print('theoretical FPT_0.95,9arm = C*R/PCT\n', 9/(Td*.95))
# print('theoretical FPT_1arm = C*R/PCT\n', 1*100/(Td*FPE_all_array))
# print('theoretical FPT_9arm = C*R/PCT\n', 9*100/(Td*FPE_all_array))
# print()
# print('PCT_1arm = 1/FPT\n', 1/FPT_all_array)
# print('PCT_9arm = 1/FPT\n', 1/(FPT_all_array*9))
# print()

# print('FPE\n', FPE_all_array)
# print('FPT\n', FPT_all_array)
# print('V\n', V_all_array)



The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR11_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR11_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR13_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR13_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR23_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR23_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR33_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR33_Vbest.csv
FPT
 [[0.316 0.249 0.253 0.300 0.378 0.380 0.360 0.324 0.396 0.307 0.389 0.193
  0.160 0.200 0.191 0.227 0.244 0.302 0.229 0.209 0.200 0.338 0.267 0.207
  0.247

In [15]:
# # line plots with mean rolling FPT, V, and N

# plotting_variable = 'FPE'

# x = range(len(file_name_CR))

# mean_colors = [bright_color_array[2], bright_color_array[5]] 

# fig, ax = plt.subplots(1, 1)

# sns.lineplot(ax=ax, data=df_fcfs, x="C/R/C*R", y=plotting_variable, color=mean_colors[0])
# sns.lineplot(ax=ax, data=df_goal, x="C/R/C*R", y=plotting_variable, color=mean_colors[1])

# ax.legend(handles=ax.lines[:], labels=["FCFS","GP"],bbox_to_anchor=(1., 1.),loc='best', fontsize=9)


# ax.set_xlabel('No. columns/rows/arms')
# ax.set_ylabel('Mean FPE (%)')

# plt.xticks([0, 1, 2, 3], ["1/1/1", "1/3/3", "2/3/6", "3/3/9"])
# # ax.set_xticks(x, file_name_CR, rotation='vertical')
# # ax.set_xticklabels(tested_CRA_rep, rotation=45, fontsize=8)

# fig.tight_layout()

# file_name = './plots/all_sched_' + sched_algo[0] + '_' + sched_algo[1] + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300)

# plt.show()

In [50]:
# #### Plot boxplot comparison of scheduling algorithm

# n_subplots = 3

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# # Now fill the boxes with desired colors
# box_colors = [pale_color_array[1], pale_color_array[3]]
# mean_colors = [bright_color_array[2], bright_color_array[5]] 
# num_boxes = len(tested_CRA_rep)
# medians = np.empty(num_boxes)

# ############################## FPE ##################################
# bp = axs[0].boxplot(FPE_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between Dark Khaki and Royal Blue
#     axs[0].add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs[0].plot(median_x, median_y, 'r', linewidth=2)
#     medians[i] = median_y[0]
#     # Finally, overplot the sample averages, with horizontal alignment
#     # in the center of each box
# #     axs[0].plot(np.average(med.get_xdata()), np.average(FPE_all_array.T[i]),
# #              color=mean_colors[i % 2], marker='o', markeredgecolor='k')

# y_95_line = np.ones(len(tested_CRA_rep))*95
# x_95_line = np.arange(1, len(tested_CRA_rep)+1)
# print(x_95_line)
# axs[0].plot(x_95_line , y_95_line, color='r', marker=' ', linestyle='--', linewidth=1, label='95% FPE threshold')


# axs[0].set_xlim(0.5, num_boxes + 0.5)
# axs[0].set_xticklabels([ ])
# axs[0].set_ylabel('FPE (%)')

# ############################## FPT ##################################
# bp = axs[1].boxplot(FPT_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between colors depending on data
#     axs[1].add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs[1].plot(median_x, median_y, color='r', linewidth=2)
#     medians[i] = median_y[0]
#     # Finally, overplot the sample averages, with horizontal alignment
#     # in the center of each box
# #     axs[1].plot(np.average(med.get_xdata()), np.average(FPT_all_array.T[i]),
# #              color=mean_colors[i % 2], marker='o', markeredgecolor='k')

# axs[1].set_ylabel('FPT (fruits/s)')


# ############################## V ##################################
# bp = axs[2].boxplot(V_all_array.T, showmeans=True, notch=False, vert=True, whis=1.5, showfliers=False)
# plt.setp(bp['boxes'], color='black')
# plt.setp(bp['whiskers'], color='black')
# # plt.setp(bp['fliers'], color='red', marker='+')

# for i in range(num_boxes):
#     box = bp['boxes'][i]
#     box_x = []
#     box_y = []
#     for j in range(5):
#         box_x.append(box.get_xdata()[j])
#         box_y.append(box.get_ydata()[j])
#     box_coords = np.column_stack([box_x, box_y])
#     # Alternate between colors depending on data
#     axs[2].add_patch(Polygon(box_coords, facecolor=box_colors[i % 2]))
#     # Now draw the median lines back over what we just filled in
#     med = bp['medians'][i]
#     median_x = []
#     median_y = []
#     for j in range(2):
#         median_x.append(med.get_xdata()[j])
#         median_y.append(med.get_ydata()[j])
#         axs[2].plot(median_x, median_y, color='r', linewidth=2)
#     medians[i] = median_y[0]
#     # Finally, overplot the sample averages, with horizontal alignment
#     # in the center of each box
# #     axs[1].plot(np.average(med.get_xdata()), np.average(FPT_all_array.T[i]),
# #              color=mean_colors[i % 2], marker='o', markeredgecolor='k')

# axs[2].set_ylabel('V (cm/s)')

# # remove y-ticks from FPT
# axs[1].set_xticklabels([ ])

# axs[-1].set_xlabel('No. columns/rows/arms')
# axs[-1].set_xlim(0.5, num_boxes + 0.5)
# axs[-1].set_xticklabels(tested_CRA_rep,
#                     rotation=45, fontsize=8)

# fig.tight_layout()


# # Finally, add a basic legend
# fig.text(0.15, 0.68-.05, 'FCFS                   ',
#          backgroundcolor=box_colors[0], color='black', weight='roman',
#          size='x-small')
# fig.text(0.15, 0.645-.05, 'Goal Programing',
#          backgroundcolor=box_colors[1], color='black', weight='roman', 
#          size='x-small')
    
# file_name = './plots/boxplot_sched_' + sched_algo[0] + '_' + sched_algo[1] + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300)
    
# plt.show()

<IPython.core.display.Javascript object>

[1 2 3 4 5 6 7 8]
Saving plot of FPT and FPE vs density in ./plots/boxplot_sched_fcfs_goal.pdf


## T-tests

In [30]:
# # for t-tests printed out as latex table

# # latex table heading
# # print as latex table

# print('\\begin{table}')
# print('\\centering')
# print('\\begin{tabular}{|c |c c |c c|}')
# print('\\multicolumn{5}{c}{ scheduling algorithm =', sched_algo, '} \\\\')
# print('\\hline')
# print('\\multicolumn{1}{|c|}{ } & \\multicolumn{2}{|c|}{FPE} & \\multicolumn{2}{|c|}{FPT} \\\\')
# print('C/R/C*R & p-value & Decision & p-value & Decision \\\\')
# print('\\hline')

# cr_index_all = 0
# cr_index     = 0

# # explanation: https://www.jmp.com/en_us/statistics-knowledge-portal/t-test/two-sample-t-test.html
# # scipy.stats: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# # for cr_index in range(len(tested_CRA)):
# for index in file_name_CR:
#     # perform an individual t-test for FPE and FPE for every C/R/C*R combo
#     fpe_t = stats.ttest_ind(FPE_all_array[cr_index_all,:], FPE_all_array[cr_index_all+1,:], equal_var=False, nan_policy='omit')
#     fpt_t = stats.ttest_ind(FPT_all_array[cr_index_all,:], FPT_all_array[cr_index_all+1,:], equal_var=False, nan_policy='omit')
    
#     if fpe_t.pvalue < 0.05:
# #         print('         ** Reject the null, there is a difference in the mean FPE **')
#         reject_fpe = 'Reject the null'
#     else:
#         reject_fpe = 'Accept the null'
        
#     if fpt_t.pvalue < 0.05:
# #         print('         ** Reject the null, there is a difference in the mean FPT **')
#         reject_fpt = 'Reject the null'
#     else:
#         reject_fpt = 'Accept the null'
           
#     print(f'%s & %2.2f & %s & %2.2f & %s \\\\' %(tested_CRA[cr_index],fpe_t.pvalue, reject_fpe, fpt_t.pvalue, reject_fpt))
    
#     cr_index_all += 2
#     cr_index += 1
    
# # finish the tabular format
# print('\\hline')
# print('\\end{tabular}')
# print('\\caption{Results of the t-test comparing the FPE and FPT means of using FCFS with ESS versus max FPE MIP with ESS for all configurations when the ``best'' harvester speed is chosen and rows are partitioned by fruits.}')
# print('\\label{table:ttest_%s%s}' %(sched_algo[0],sched_algo[1]))
# print('\\end{table}')



## Plot the relationship between chosen V, FPT, and the number of fruits in a segment

In [11]:
# # plot the chosen V against the number of fruits and the FPT against number of fruits
# # mean_colors = [bright_color_array[1], bright_color_array[4]] 

# # CR = index: 11 = 1, 13 = 3, 23 = 5, 33 = 7

# fruit_i = 7
# height_i = fruit_i+1

# fig, axs = plt.subplots(2,1)

# axs[0].grid(True)
# axs[1].grid(True)

# axs[0].scatter(N_segment, V_all_array[fruit_i,:], color=mean_colors[1], label='Goal Programming')
# # axs[0].plot(N, pred_r_plot[int(fruit_i/2)][0], color=pred_colors[0], label='predicted')
# axs[0].set_ylabel('best V (cm/s)')

# axs[1].scatter(N_segment, FPT_all_array[fruit_i,:], color=mean_colors[1])
# # axs[1].plot(N, pred_r_plot_fpt[int(fruit_i/2)][0], color=pred_colors[0], label='predicted')
# axs[1].set_ylabel('FPT (fruit/s)')

# # axs[0,1].scatter(N_segment, V_all_array[height_i,:], color=mean_colors[1], label='partition by height')
# # axs[1,1].scatter(N_segment, FPT_all_array[height_i,:], color=mean_colors[1])
# # axs[2,1].scatter(N_segment, FPT_all_array[height_i,:], color=mean_colors[1])

# axs[1].set_xlabel('Number of fruits in segment')

# # axs[0].title.set_text('partition by fruit')

# # fig.text(0.70, 0.68-.03, 'Row height equal fruits  ',
# #          backgroundcolor=box_colors[0], color='black', weight='roman',
# #          size='x-small')

# # # curve's equation on the plot
# # fig.text(.77, .9, r"$ \frac{k}{N} $= $ \frac{%0.3f}{N} $" %popt[0], backgroundcolor='w')
# # # line's equation on the plot
# # fig.text(.75, .42, r"$ m*N $= $ %0.3f * N $" %popt_fpt[0], backgroundcolor='w', size='small')

# # axs[0].title.set_text('FCFS')
# # axs[1].title.set_text('Goal')

# axs[0].set_ylim(-3, 100)
# axs[1].set_ylim(-0.3, 3)
# fig.tight_layout()

# file_name = './plots/N_vs_sched_CR' + str(file_name_CR[int(fruit_i/2)]) + '.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

## Line plot comparing all scheduling algorithm means

In [16]:
# # read a CSV file comparing V FPE and FPT for each C/R/C*R setting

partition_by = 'fruit'

print('Partitioning rows by %s' %partition_by)

# number of fruits equal
FPE_array_fruits = np.zeros((len(tested_CRA), runs_in_file, len(sched_algo)))
FPT_array_fruits = np.zeros((len(tested_CRA), runs_in_file, len(sched_algo)))
V_array_fruits   = np.zeros((len(tested_CRA), runs_in_file, len(sched_algo)))
T_array_fruits   = np.zeros((len(tested_CRA), runs_in_file, len(sched_algo))) # time to solve


FPE_avg_array_fruits = np.zeros((len(tested_CRA), len(sched_algo)))
FPE_std_array_fruits = np.zeros((len(tested_CRA), len(sched_algo)))
FPT_avg_array_fruits = np.zeros((len(tested_CRA), len(sched_algo)))
FPT_std_array_fruits = np.zeros((len(tested_CRA), len(sched_algo)))
V_avg_array_fruits   = np.zeros((len(tested_CRA), len(sched_algo)))
V_std_array_fruits   = np.zeros((len(tested_CRA), len(sched_algo)))
T_avg_array_fruits   = np.zeros((len(tested_CRA), len(sched_algo)))
T_std_array_fruits   = np.zeros((len(tested_CRA), len(sched_algo)))

cr_index = 0

# only column increase combinations
for cr in file_name_CR:
    for algo_i in range(len(sched_algo)):
        file_name = folder + date+'_'+sched_algo[algo_i]+'_2022_data_fruit_CR' + str(cr) + '_Vbest' + file_type
        print('\nThe file being opened is', file_name) 

        with open(file_name) as csvfile:

            reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            row_i = 0

            for row in reader:
                if row_i < runs_in_file:
                    FPE_array_fruits[cr_index,row_i, algo_i] = row[9]
                    FPT_array_fruits[cr_index,row_i, algo_i] = row[11]
                    V_array_fruits[cr_index,row_i, algo_i] = row[7]
                    T_array_fruits[cr_index,row_i, algo_i] = row[8]
#                 print('By fruit: for V %s, row %d' %(V[V_i], row_i))
#                 print('FPE value %3.3f and FPT value %3.3f' %(row[8], row[10]))

                row_i += 1

        # all files for this V have been processed, calculate the average and standard deviation for them
        FPE_avg_array_fruits[cr_index,algo_i] = np.average(FPE_array_fruits[cr_index,:,algo_i])
        FPE_std_array_fruits[cr_index,algo_i] = np.std(FPE_array_fruits[cr_index,:,algo_i])
        FPT_avg_array_fruits[cr_index,algo_i] = np.average(FPT_array_fruits[cr_index,:,algo_i])
        FPT_std_array_fruits[cr_index,algo_i] = np.std(FPT_array_fruits[cr_index,:,algo_i])
        V_avg_array_fruits[cr_index,algo_i] = np.average(V_array_fruits[cr_index,:,algo_i])
        V_std_array_fruits[cr_index,algo_i] = np.std(V_array_fruits[cr_index,:,algo_i])
        T_avg_array_fruits[cr_index,algo_i] = np.average(T_array_fruits[cr_index,:,algo_i])
        T_std_array_fruits[cr_index,algo_i] = np.std(T_array_fruits[cr_index,:,algo_i])

    cr_index += 1

    
print('\n\nFor equal fruit')
print('------------------------------------------------------')
print('Average FPE values:\n', FPE_avg_array_fruits, '\n')  
print('Standard dev FPE values:\n', FPE_std_array_fruits, '\n')  
print('Average FPT values:\n', FPT_avg_array_fruits, '\n')  
print('Standard dev FPT values:\n', FPT_std_array_fruits, '\n') 
print('Average T values:\n', T_avg_array_fruits, '\n')  
print('Standard dev T values:\n', T_std_array_fruits) 


Partitioning rows by fruit

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR11_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR11_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR13_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR13_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR23_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR23_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR33_Vbest.csv

The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR33_Vbest.csv


For equal fruit
------------------------------------------------------
Average FPE values:
 [[99.672 93.407]
 [96.247 92.794]
 [96.

In [22]:
# # ## Line subplots

# n_subplots = 3

# line_style = '-'
# line_width = 2
# cap_size   = 3

# y_95_line = np.ones(len(x))*95

# fig, axs = plt.subplots(n_subplots, 1)
# ## want subplots to stack in columns so they can be compared, see
# ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# axs[0].plot(x , y_95_line, color='r', marker=' ', linestyle='--', linewidth=1, label='95% FPE threshold')

# for a_i in range(len(sched_algo)):
#     axs[0].plot(x , FPE_avg_array_fruits[:,a_i], color=mean_colors[a_i], marker='s', markersize=5, linestyle=line_style, linewidth=line_width, label='algorithm = ' + sched_algo[a_i])
#     #     axs[0].errorbar(x , FPE_avg_array_fruits[:,V_i], yerr=FPE_std_array_fruits[:,V_i], color=bright_color_array[V_i], linestyle=line_style, linewidth=line_width, capsize=cap_size)

#     ## Labels and grid for ax0
#     axs[0].set_ylabel('mean FPE (%)')
#     axs[0].grid(True)
#     axs[0].set_xticklabels([ ])


#     axs[1].plot(x, FPT_avg_array_fruits[:,a_i], color=mean_colors[a_i], marker='s', markersize=5, linestyle=line_style, linewidth=line_width)
#     axs[1].grid(True)
#     axs[1].set_ylabel('mean FPT (fruits/s)')
#     axs[1].set_xticklabels([ ])


#     axs[2].plot(x , V_avg_array_fruits[:,a_i], color=mean_colors[a_i], marker='s', markersize=5, linestyle=line_style, linewidth=line_width, label='algorithm = ' + sched_algo[a_i])
#     axs[2].set_ylabel('mean V (cm/s)')
#     axs[2].grid(True)
#     axs[2].set_xticklabels([ ], fontsize=8)


# #     axs[1,1].plot(x , T_avg_array_fruits[:,a_i], color=mean_colors[a_i], marker='s', markersize=5, linestyle=line_style, linewidth=line_width, label='algorithm = ' + sched_algo[a_i])
# #     axs[1,1].grid(True)
# #     axs[1,1].set_ylabel('mean solve time (s)')

# legend = axs[0].legend(bbox_to_anchor=(1., 1.),loc='best', fontsize=9)

# axs[-1].set_xlabel('No. columns/rows/arms')

# # plt.xticks(x, tested_CRA)#, rotation='vertical')
# axs[-1].set_xticks(x, tested_CRA, rotation=45)

# axs[0].set_ylim(91, 100)
# axs[1].set_ylim(-0.2, 2.8)
# axs[2].set_ylim(-0.3, 45)

# fig.tight_layout()

# file_name = './plots/all_v_sched.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300)
# plt.show()

<IPython.core.display.Javascript object>

Saving plot of FPT and FPE vs density in ./plots/all_v_sched.pdf


## Calc, make a table, and plot the mean running time

### open files and make dataframes

In [51]:
plotting_variable = 'OR_FPE'

file_name_CR = [11, 13, 23, 33]
tested_CRA = np.array(['1/1/1', '1/3/3', '2/3/6', '3/3/9'])

list_df = list()

# The file being opened is ./csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR11_Vbest.csv
# The file being opened is ./csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR11_Vbest.csv

for cr_index in range(len(file_name_CR)):
    file_fcfs = './csv_files/20240211_algo_compare/20240211_fcfs_2022_data_fruit_CR' + str(file_name_CR[cr_index]) 
    file_goal = './csv_files/20240211_algo_compare/20240211_goal_2022_data_fruit_CR' + str(file_name_CR[cr_index]) 
#     print('file being opened: \n', file)

    ## DataFrames
    df_fcfs = pd.read_csv(file_fcfs + '_Vbest.csv')
    df_goal = pd.read_csv(file_goal + '_Vbest.csv')

    # add column titles
    df_fcfs.columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V',
                      'run_time', 'OR_FPE', 'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d']
    df_goal.columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V',
                      'run_time', 'OR_FPE', 'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d']

    df_fcfs['algorithm']='FCFS'
    df_goal['algorithm']='GP'

    df_sched = pd.concat([df_fcfs, df_goal], ignore_index=True)

    # make Col and R ints
    df_sched['Col'] = df_sched['Col'].astype(int)
    df_sched['Row'] = df_sched['Row'].astype(int)
    # create a 'C/R/C*R' column
    df_sched['C/R/C*R']= tested_CRA[cr_index]   
    
    list_df.append(df_sched)   

# hue_cr = df[['Col', 'Row']].apply(lambda row: f"{row.Col}, {row.Row}", axis=1)
# hue_cr.name = 'C, R'

# print(list_df_SW[0])
df_concat = pd.concat([list_df[0], list_df[1], list_df[2], list_df[3]], ignore_index=True)

### Plot mean running times

In [52]:
# Use seaborn to plot the mean running time

# columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V', 'run_time', 'OR_FPE',
#                   'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d', 'D_size', 'algorithm', 'C/R/C*R']


# sns.set_style("white")
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette([bright_color_array[2], bright_color_array[5]]))

# ax = sns.catplot(data=df_concat, x='C/R/C*R', y='run_time', kind='swarm', hue='algorithm', s=3)

ax = sns.lineplot(data=df_concat, x='C/R/C*R', y='run_time', hue='algorithm', style='algorithm', markers=True, legend=True)



# set the labels
ax.set(xlabel='no. columns/rows/arms', ylabel='mean running time (s)')

# sns.lineplot(ax=ax, data=list_df_OR[0], x='C/R/C*R', y=plotting_variable)
# sns.lineplot(ax=ax, data=list_df_OR[1], x='C/R/C*R', y=plotting_variable)
# sns.lineplot(ax=ax, data=list_df_OR[2], x='C/R/C*R', y=plotting_variable)
# sns.lineplot(ax=ax, data=list_df_OR[3], x='C/R/C*R', y=plotting_variable)

file_name = './plots/time_v_sched.pdf'
print('Saving plot of FPT and FPE vs density in', file_name)
plt.savefig(file_name,dpi=300, bbox_inches='tight')

plt.show()

<IPython.core.display.Javascript object>

Saving plot of FPT and FPE vs density in ./plots/time_v_sched.pdf


### Create Latex table with mean running time

In [74]:
# # create the two ANOVA tables printed out as latex table
# tested_CRA = np.array(['1/1/1', '1/3/3', '2/3/6', '3/3/9'])

# # result = df_concat.groupby(['C/R/C*R'], as_index=False).agg({'run_time':['mean','std'],'b':'first', 'd':'first'})
# # print(result)
# result = df_concat.groupby(['C/R/C*R', 'algorithm'], as_index=False).agg({'run_time':['mean','std']})
# # print(result, '\n')
# # print(result.iloc[0]['run_time']['mean'])


# # latex table heading
# # print as latex table
# print('\\begin{table}')
# print('\\centering')
# print('\\resizebox{0.9\\columnwidth}{!}{\\begin{tabular}{|c |c c c c|}')
# print('\\multicolumn{5}{c}{ Mean running time for each algorithm, in seconds } \\\\')
# print('\\hline')
# print('\\multicolumn{1}{|c|}{ Algorithm } & \\multicolumn{4}{|c|}{ C/R/C*R } \\\\')
# print(' & 1/1/1 & 1/3/3 & 2/3/6 & 3/3/9 \\\\')
# print('\\hline')

# for algo_index in range(2):
#     if algo_index == 0:
#         algo_use = 'FCFS'
#     else:
#         algo_use = 'GP'
        
#     print(f'%s & %.2f +/- %.2f & %.2f +/- %.2f & %.2f +/- %.2f & %.2f +/- %.2f \\\\' %(algo_use, 
#                                                                                result.iloc[algo_index]['run_time']['mean'],
#                                                                                result.iloc[algo_index]['run_time']['std'], 
#                                                                                result.iloc[algo_index+2]['run_time']['mean'],
#                                                                                result.iloc[algo_index+2]['run_time']['std'],
#                                                                                result.iloc[algo_index+4]['run_time']['mean'],
#                                                                                result.iloc[algo_index+4]['run_time']['std'],
#                                                                                result.iloc[algo_index+6]['run_time']['mean'],
#                                                                                result.iloc[algo_index+6]['run_time']['std']))

# # finish the tabular format
# print('\\hline')
# print('\\end{tabular}}')
# print('\\caption{Table showing the running time when using First Come First Served versus Goal Programming for the four harvester configurations. Running time is in seconds.}')
# print('\\label{table:run_time_algo}')
# print('\\end{table}')


\begin{table}
\centering
\resizebox{0.65\columnwidth}{!}{\begin{tabular}{|c |c c c c|}
\multicolumn{5}{c}{ Mean running time for each algorithm in seconds } \\
\hline
\multicolumn{1}{|c|}{ Algorithm } & \multicolumn{4}{|c|}{ C/R/C*R } \\
\hline
 & 1/1/1 & 1/3/3 & 2/3/6 & 3/3/9 \\
\hline
FCFS & 0.00 +/- 0.00 & 0.00 +/- 0.00 & 0.01 +/- 0.00 & 0.02 +/- 0.00 \\
GP & 235.95 +/- 272.82 & 32.28 +/- 14.19 & 472.10 +/- 237.11 & 335.59 +/- 291.86 \\
\hline
\end{tabular}}
\caption{Table showing the running time when using First Come First Served versus Goal Programming for each harvester configuration. Running time is in seconds.}
\label{table:run_time_algo}
\end{table}


# Tests to determine if solving whole orchards works

In [34]:
# # CSV data
# # folder name
# folder = './csv_files/20240221_sliding/'

# date   = '20240221'

# tested_CRA = np.array(['3/3/9'])
# tested_CRA_rep = np.array(['3/3/9', '3/3/9'])
# # as ints for looping
# file_name_CR = [33]

# d_plan = 3.5 # in m
# D      = np.array([1, 1/2, 1/3]) # with respect to d_plan(ning window)

# tested_CRA = np.array(['1/1/1', '1/3/3', '2/3/6', '3/3/9'])#'1/1/1', '1/2/2', '2/1/2', '1/3/3', '3/1/3', '2/2/4', '2/3/6', '3/2/6', '3/3/9'])
# tested_CRA_rep = np.array(['1/1/1','1/1/1', '1/3/3', '1/3/3', '2/3/6', '2/3/6','3/3/9', '3/3/9'])#'1/1/1','1/1/1', '1/2/2','1/2/2', '2/1/2','2/1/2', '1/3/3','1/3/3', '3/1/3','3/1/3', '2/2/4','2/2/4', '2/3/6','2/3/6', '3/2/6','3/2/6', '3/3/9','3/3/9'])
# # as ints for looping
# # file_name_CR = [11, 12, 21, 13, 31, 22, 23, 32, 33]
# file_name_CR = [11,13,23,33]

# max_seg = 15 # in m, max "orchard row" length

# sched_algo = 'goal'

# # csv file ending
# file_type = '.csv'

# test_number = 2 # can be 1, 2, or 3 for 1/1 d_v, 1/2 d_v, 1/3 d_v

# # colors
# # see https://personal.sron.nl/~pault/#sec:qualitative
# # and a different ben (https://stats.stackexchange.com/users/9311/a-different-ben), "Best" series of colors to use for differentiating series in publication-quality plots, URL (version: 2021-03-03): https://stats.stackexchange.com/q/118259
# bright_color_array = ['#228833', '#4477AA', '#66CCEE', '#AA3377', '#EE6677', '#CCBB44', '#BBBBBB']
# pale_color_array   = ['#BBCCEE', '#CCEEFF', '#CCDDAA', '#EEEEBB', '#FFCCCC', '#DDDDDD']
# dark_color_array   = ['#222255', '#225555', '#225522', '#666633', '#663333', '#555555']


In [35]:
# # all runs use partition by fruits and attempt to find the best V (range 1-20 cm/s)

# FPE_all_list = list()
# FPT_all_list = list()
# V_all_list   = list()
# # Td_all_array  = np.zeros([len(tested_CRA)*2, runs_in_file])
# N_all_list    = list()

# FPE_rolling_list = list()
# FPT_rolling_list = list()

# # only look at C/R combinations in the file_name_CR array
# if test_number == 1:
#     file_name = folder + date+'_'+sched_algo+'_2022_data_11' + file_type
# elif test_number == 2:
#     file_name = folder + date+'_'+sched_algo+'_2022_data_11[0]' + file_type
# else:
#     file_name = folder + date+'_'+sched_algo+'_2022_data_11[1]' + file_type
    
# print('\nThe file being opened is', file_name) 

# file_n = 0

# with open(file_name) as csvfile:
#     FPE_all_list.append([]) 
#     FPT_all_list.append([]) 
#     V_all_list.append([])   
#     N_all_list.append([]) 
#     FPE_rolling_list.append([])
#     FPT_rolling_list.append([])

#     reader = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
#     row_i = 0

#     for row in reader:
        
#         if row[0] != file_n: # new file
#             FPE_all_list.append([]) 
#             FPT_all_list.append([]) 
#             V_all_list.append([])   
#             N_all_list.append([])  
#             FPE_rolling_list.append([])
#             FPT_rolling_list.append([])
            
#             file_n += 1
            
#         FPE_all_list[file_n].append(row[9]) 
#         FPT_all_list[file_n].append(row[11]) 
#         V_all_list[file_n].append(row[7])   
#         N_all_list[file_n].append(row[3]) 
        
#         FPE_rolling_list[file_n].append(row[10])
#         FPT_rolling_list[file_n].append(row[12])
        
        
# #         # save the number of fruits in each array available to the arms (if in dead space, not counted here)
# #         N_segment[row_i] = row[3]
# #         # FCFS
# #         FPE_all_array[cr_index_all,row_i] = row[9]
# #         FPT_all_array[cr_index_all,row_i] = row[11]
# #         V_all_array[cr_index_all,row_i] = row[7]
# #                     Td_all_array[cr_index_all,row_i] = row[13]

# #                     print('By fruit: for V %s, row %d' %(V[V_i], row_i))
# #                     print('FPE value %3.3f and FPT value %3.3f' %(row[8], row[10]))


#         row_i += 1

    
# # print(FPE_rolling_list)
    

# # print('FPE\n', FPE_all_array)
# # print('FPT\n', FPT_all_array)
# # print('V\n', V_all_array)

In [8]:
plotting_variable = 'OR_FPT'

file_name_CR = [11, 13, 23, 33]
tested_CRA = np.array(['1/1/1', '1/3/3', '2/3/6', '3/3/9'])

list_df_SW = list()
list_df_OR = list()

for cr_index in range(len(file_name_CR)):
    file = './csv_files/20240224_sliding/20240224_fcfs_2022_data_' + str(file_name_CR[cr_index]) 
#     print('file being opened: \n', file)

    ## DataFrame for rolling values
    dp1 = pd.read_csv(file + '.csv')
    dp2 = pd.read_csv(file + '[0].csv')
    dp3 = pd.read_csv(file + '[1].csv')

    # add column titles
    dp1.columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V',
                      'run_time', 'OR_FPE', 'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d']
    dp2.columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V',
                      'run_time', 'OR_FPE', 'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d']
    dp3.columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V',
                      'run_time', 'OR_FPE', 'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d']

    # add a column that indicates the size of D
    dp1['D_size']='1*dw'
    dp2['D_size']='1/2*dw'
    dp3['D_size']='1/3*dw'
    
    if cr_index <= 1:
        D = 1.2
    elif cr_index < 3:
        D = 2.35
    else:
        D = 3.5
    
    dp1['segment_start'] = dp1['segment'] * D
    dp2['segment_start'] = dp2['segment'] * D/2
    dp3['segment_start'] = dp3['segment'] * D/3

    df_SW = pd.concat([dp1, dp2, dp3], ignore_index=True)

    # make Col and R ints
    df_SW['Col'] = df_SW['Col'].astype(int)
    df_SW['Row'] = df_SW['Row'].astype(int)
    # create a 'C/R/C*R' column
    df_SW['C/R/C*R']= tested_CRA[cr_index]

    # add a column that indicates what algorithm was used based on N
    # init by copying the 'N' column
    df_SW['algorithm'] = df_SW['N']
    # change values accoring to the value of N in the new 'algorithm' column
    df_SW.loc[df_SW['N'] > 20, 'algorithm'] = 'goal'
    df_SW.loc[df_SW['N'] <= 20, 'algorithm'] = 'fcfs'
    
    
    list_df_SW.append(df_SW)
    
    
    # need to get the global FPE for the max segment in each file, an update lower down talks about changes needed
    # see http://stackoverflow.com/a/21007047/778533
    sorted = dp1.sort_values(by='segment')
    dp1_OR = sorted.drop_duplicates('file', keep='last')

    sorted = dp2.sort_values(by='segment')
    dp2_OR = sorted.drop_duplicates('file', keep='last')

    sorted = dp3.sort_values(by='segment')
    dp3_OR = sorted.drop_duplicates('file', keep='last')


    df_OR = pd.concat([dp1_OR, dp2_OR, dp3_OR], ignore_index=True)
    
    # make Col and R ints
    df_OR['Col'] = df_OR['Col'].astype(int)
    df_OR['Row'] = df_OR['Row'].astype(int)
    # create a 'C/R/C*R' column
    df_OR['C/R/C*R']= tested_CRA[cr_index]

    # add a column that inidcates what algorithm was used based on N
    # init by copying the 'N' column
    df_OR['algorithm'] = df_OR['N']
    # change values accoring to the value of N in the new 'algorithm' column
    df_OR.loc[df_OR['N'] > 20, 'algorithm'] = 'goal'
    df_OR.loc[df_OR['N'] <= 20, 'algorithm'] = 'fcfs'
    
    list_df_OR.append(df_OR)
    

# hue_cr = df[['Col', 'Row']].apply(lambda row: f"{row.Col}, {row.Row}", axis=1)
# hue_cr.name = 'C, R'

# print(list_df_SW[0])


In [9]:
# ## DataFrame for cumulative values

# for cr_index in range(len(file_name_CR)):
#     # need to get the global FPE for the max segment in each file, an update lower down talks about changes needed
#     # see http://stackoverflow.com/a/21007047/778533
#     sorted = dp1.sort_values(by='segment')
#     dp1_OR = sorted.drop_duplicates('file', keep='last')

#     sorted = dp2.sort_values(by='segment')
#     dp2_OR = sorted.drop_duplicates('file', keep='last')

#     sorted = dp3.sort_values(by='segment')
#     dp3_OR = sorted.drop_duplicates('file', keep='last')


#     df_OR = pd.concat([dp1_OR, dp2_OR, dp3_OR], ignore_index=True)
    
#     # make Col and R ints
#     df_OR['Col'] = df_OR['Col'].astype(int)
#     df_OR['Row'] = df_OR['Row'].astype(int)
#     # create a 'C/R/C*R' column
#     df_OR['C/R/C*R']= tested_CRA[cr_index]

#     # add a column that inidcates what algorithm was used based on N
#     # init by copying the 'N' column
#     df_OR['algorithm'] = df_OR['N']
#     # change values accoring to the value of N in the new 'algorithm' column
#     df_OR.loc[df_OR['N'] > 20, 'algorithm'] = 'goal'
#     df_OR.loc[df_OR['N'] <= 20, 'algorithm'] = 'fcfs'
    
#     list_df_OR.append(df_OR)

# print(list_df_OR[0])
# print()
# print(list_df_OR[3])

In [10]:
# # Setup for stats tests. Cumulative should only look at end points, rolling at all points

# # columns = ['file', 'segment',  'D_size', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V',
# #                   'run_time', 'FPE_cumulative', 'FPE_rolling', 'FPT_cumulative', 'FPT_rolling', 'T_d']

# if plotting_variable == 'OR_FPT' or plotting_variable == 'OR_FPT':
#     df = df_OR
# else:
#     df = df_SW
    
# ## Show information on the dataframe being used
# # df.info()

# ## get the overall mean, sd, etc for the variable being tested
# # rp.summary_cont(df['FPT_rolling'])
# # How to get the specific values from above command (T has all of them, just need to figure out indexing)
# print('The overall average rolling FPT is %0.3f 95percent Cl(%0.2f, %0.2f)\n' 
#       %(rp.summary_cont(df[plotting_variable]).Mean[0],
#         rp.summary_cont(df[plotting_variable]).T[0][5],
#         rp.summary_cont(df[plotting_variable]).Interval[0]))

# # summerizes the means, sd, etc when divided by groupby (so the mean, std, etc when D=1, D=2, ...)
# print('--------------------------------------------------------------------------------------------------')
# print('summary of the the mean, sd, etc when the dependent variable is grouped by independent variable')
# print(rp.summary_cont(df[plotting_variable].groupby(df['D_size'])))
# print('--------------------------------------------------------------------------------------------------\n')


# # init the lists to save the statistics for ANOVA and Games-Howell
# anova_F    = list()
# anova_pval = list()
# anova_dec  = list()

# games_T    = [[],[],[]]
# games_pval = [[],[],[]]
# games_dec  = [[],[],[]]


In [11]:
# # Test the Normality Assumption (not 100% necessary here)

# test_string = str(plotting_variable) + ' ~ C(D_size)'
# # test the normality assumption using the Shapiro-Wilks test with output (W-test statistic, p-value).
# model = ols(test_string, data=df).fit()

# shap_stats = stats.shapiro(model.resid)
# print('Shapiro test of normality assumption,\n     p>0.05 means Normal (good)\n    ', shap_stats, '\n')

# if shap_stats.pvalue < 0.05:
#     print('Normality assumption is not met')

# else:
#     print('Normality assumption applies')

# fig = plt.figure()
# ax = fig.add_subplot(111)

# normality_plot, stat = stats.probplot(model.resid, plot= plt, rvalue= True)
# ax.set_title("Probability plot of model residual's", fontsize= 20)
# ax.set

# plt.show()

In [12]:
# Test homogeneity of variance assumption

# for cr_index in range(len(file_name_CR)):

#     bart_stats = stats.bartlett(list_df_OR[cr_index][plotting_variable][list_df_OR[cr_index]['D_size'] == '1*dp'], 
#                                 list_df_OR[cr_index][plotting_variable][list_df_OR[cr_index]['D_size'] == '1/2*dp'], 
#                                 list_df_OR[cr_index][plotting_variable][list_df_OR[cr_index]['D_size'] == '1/3*dp'])

#     print('Bartlett test of homogeneity of variance,\n     p>0.05 means not significant (good)\n    ', bart_stats, '\n')



# test homogeneity of variance
# null hypothesis that all input samples are from populations with equal variances
# lev_stats = stats.levene(df[plotting_variable][df['D_size'] == '1*dp'], 
#                          df[plotting_variable][df['D_size'] == '1/2*dp'], 
#                          df[plotting_variable][df['D_size'] == '1/3*dp'])

# print('Leven test of homogeneity of variance,\n     p>0.05 means not significant (good)\n    ', lev_stats, '\n')


# if bart_stats.pvalue < 0.05 or lev_stats.pvalue < 0.05: 
#     print('Homogeneity assumption is not met, use Welch\'s ANOVA\n')
    
# else:
#     print('Homogeneity assumption is met, use normal ANOVA\n')


# fig = plt.figure()
# ax = fig.add_subplot(111)

# ax.set_title("Box Plot of rolling FPT by D_size", fontsize= 20)
# ax.set

# data = [df[plotting_variable][df['D_size'] == '1*dp'],
#         df[plotting_variable][df['D_size'] == '1/2*dp'],
#         df[plotting_variable][df['D_size'] == '1/3*dp']]

# ax.boxplot(data,
#            labels= ['D=1*d_p', 'D=1/2*d_p', 'D=1/3*d_p'],
#            showmeans= True)

# plt.xlabel("D size (m)")
# plt.ylabel("Tested variable")

# plt.show()

In [20]:
# # for ANOVA-tests 
# # see https://www.pythonfordatascience.org/anova-python/

# # init the lists to save the statistics for ANOVA and Games-Howell
# anova_F    = list()
# anova_pval = list()
# anova_dec  = list()

# games_T    = list()
# games_pval = list()
# games_dec  = list()

# for cr_index in range(len(file_name_CR)):
   
#     games_T.append([])
#     games_pval.append([])
#     games_dec.append([])
    
#     # set the correct df based on the variable being tested (plotting_variable)
#     if plotting_variable == 'OR_FPT' or plotting_variable == 'OR_FPT':
#         df = list_df_OR[cr_index]
#     else:
#         df = list_df_SW[cr_index]

#     print('Variance between groups is not equal, using Welch\'s ANOVA\n')
#     aov = pg.welch_anova(dv=plotting_variable, between='D_size', data=df)
#     print(aov)

#     if aov['p-unc'][0] < 0.05:
#         anova_dec.append('Reject the null')
#         print('\nThere is a difference between the means, using Games-Howell to determine which means are different')
#         print('In table form:')
#         games_stats = pg.pairwise_gameshowell(dv=plotting_variable, between='D_size', data=df)
#         print(games_stats)
        
#         for index in range(3):
#             games_T[cr_index].append(games_stats['T'][index])
#             games_pval[cr_index].append(games_stats['pval'][index])

#             if games_stats['pval'][index] < 0.05:
#                 games_dec[cr_index].append('Reject the null')

#             else:
#                 games_dec[cr_index].append('Accept the null')



#         if games_stats['pval'][0] < 0.05 or games_stats['pval'][1] < 0.05 or games_stats['pval'][2] < 0.05:
#             print('\nThere is a difference between at least one mean, (pval<0.05)\n')
#         else:
#             print('\nThere is NO difference between any of the means\n')            

#     else:
#         print('There is NO difference between the means\n')
#         anova_dec.append('Accept the null')

#     anova_F.append(aov['F'][0])
#     anova_pval.append(aov['p-unc'][0])

    


# # else:
# #     print('Variance between groups is equal, using normal ANOVA\n')
# #     # Perform a one-way ANOVA test
# # #     anova_out = stats.f_oneway(df[plotting_variable][df['D_size'] == 1],
# # #                                df[plotting_variable][df['D_size'] == 2],
# # #                                df[plotting_variable][df['D_size'] == 3])

# # #     print('ANOVA test,\n    ', anova_out, '\n')
# # #     aov_table = sm.stats.anova_lm(model, typ=2)
# # #     print('as a table,', aov_table)
# #     aov = pg.anova(data=df, dv='Scores', between='Group', detailed=True)
# #     print(aov)

# #     if aov['p-unc'][0] < 0.05:
# #         print('There is a difference between the means, using Tukey test to determine which means are different\n')

# #         # ## Tukey test to check if there is a difference between Ds, TRUE = there is a difference
# #         comp = mc.MultiComparison(df[plotting_variable], df['D_size'])
# #         post_hoc_res = comp.tukeyhsd()
# #         post_hoc_res.summary()

# #         # Plot the Tukey test results, if there is overlap, there is no difference between Ds
# #         # post_hoc_res.plot_simultaneous(ylabel= "D", xlabel= "Score Difference")

In [14]:
# # print out ANOVA and G.H. stats values to manually enter into table below
# print('For variable', plotting_variable)
# print('----------------------------------')
# print('ANOVA RESULTS')
# print('F values\n', anova_F)
# print('\n p-values\n', anova_pval)
# print('\n Decisions\n', anova_dec)

# print('\n\nGAMES-HOWELL RESULTS')
# print('T values\n', games_T)
# print('\n p-values\n', games_pval)
# print('\n Decisions\n', games_dec)

In [19]:
# # create the two ANOVA tables printed out as latex table
# tested_CRA = np.array(['1/1/1', '1/3/3', '2/3/6', '3/3/9'])


# # latex table heading
# # print as latex table

# print('\\begin{table}')
# print('\\centering')
# print('\\begin{tabular}{|c |c c c|}')
# print('\\multicolumn{4}{c}{ Welch\'s ANOVA for %s, $95$\\unit{\\percent} CI } \\\\' %(plotting_variable))
# print('\\hline')
# print('C/R/C*R & F & p-value & Decision \\\\')
# print('\\hline')

# for cr_index in range(len(file_name_CR)):
#     print(f'%s & %.3f & %.3f & %s \\\\' %(tested_CRA[cr_index], 
#                                               anova_F[cr_index],
#                                               anova_pval[cr_index], 
#                                               anova_dec[cr_index]))

# # finish the tabular format
# print('\\hline')
# print('\\end{tabular}')
# print('\\caption{Results from Welch\'s ANOVA to determine if the means between groups A and B are different.}')
# print('\\label{table:anova_%s}' %(plotting_variable))
# print('\\end{table}')


In [16]:
# # create the Games-Howell table printed out as latex table
                               
# # # latex table heading
# # # print as latex table

# print('\\begin{table}')
# print('\\centering')
# print('\\resizebox{1\\columnwidth}{!}{\\begin{tabular}{|c |c c c| c c c | c c c |}')
# print('\\multicolumn{10}{c}{ Games-Howell post-hoc test, $95$\\unit{\\percent} CI } \\\\')
# print('\\hline')
# print('\\multicolumn{1}{|c|}{ A and B } & \\multicolumn{3}{|c|}{$1*dp$ and $1/2*dp$} & \\multicolumn{3}{|c|}{$1*dp$ and $1/3*dp$} & \\multicolumn{3}{|c|}{$1/2*dp$ and $1/3*dp$} \\\\')
# print('C/R/C*R & F & p-value & Decision & F & p-value & Decision & F & p-value & Decision \\\\')
# print('\\hline')

# for cr_index in range(len(file_name_CR)):
#     if games_T[cr_index]:
#         # if empty is not empty
#         print(f'%s & %.3f & %.3f & %s & %.3f & %.3f & %s & %.3f & %.3f & %s \\\\' %(tested_CRA[cr_index], 
#                                                               games_T[cr_index][0],
#                                                               games_pval[cr_index][0], 
#                                                               games_dec[cr_index][0],
#                                                               games_T[cr_index][1],
#                                                               games_pval[cr_index][1], 
#                                                               games_dec[cr_index][1],
#                                                               games_T[cr_index][2],
#                                                               games_pval[cr_index][2], 
#                                                               games_dec[cr_index][2]))
    
# # finish the tabular format
# print('\\hline')
# print('\\end{tabular}}')
# print('\\caption{Results from Games-Howell post-hoc test to determine exactly which group means are different.}')
# print('\\label{table:gam_how_%s}' %(plotting_variable))
# print('\\end{table}')



In [18]:
# # Use seaborn to plot the mean OR-FPE and OR-FPT

# df_OR_concat = pd.concat([list_df_OR[0], list_df_OR[1], list_df_OR[2], list_df_OR[3]], ignore_index=True)

# # columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V', 'run_time', 'OR_FPE',
# #                   'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d', 'D_size', 'algorithm', 'C/R/C*R']


# # sns.relplot(
# #     data=df_OR,
# #     x="file", y=plotting_variable,
# #     hue="D_size", style="N", s=100)

# # sns.relplot(
# #     data=df_OR,
# #     x="segment", y=plotting_variable,
# #     hue="D_size", style="file", s=100)

# # ## want subplots to stack in columns so they can be compared, see
# # ## see https://matplotlib.org/stable/gallery/lines_bars_and_markers/cohere.html#sphx-glr-gallery-lines-bars-and-markers-cohere-py

# # sns.set_style("white")
# sns.set_style("whitegrid")

# ax = sns.catplot(data=df_OR_concat, x='C/R/C*R', y=plotting_variable, kind='swarm', hue='D_size', s=10)

# ax = sns.lineplot(data=df_OR_concat, x='C/R/C*R', y=plotting_variable, hue='D_size', legend=False)


# # set the labels
# if plotting_variable == 'OR_FPE':
#     ax.axhline(95, color='r', linestyle='--', linewidth=1)
#     ax.set(xlabel='no. columns/rows/arms', ylabel='Orchard Row FPE (%)')
    
# elif plotting_variable == 'OR_FPT':
#     ax.set(xlabel='no. columns/rows/arms', ylabel='Orchard Row FPT (fruits/s)')

# # sns.lineplot(ax=ax, data=list_df_OR[0], x='C/R/C*R', y=plotting_variable)
# # sns.lineplot(ax=ax, data=list_df_OR[1], x='C/R/C*R', y=plotting_variable)
# # sns.lineplot(ax=ax, data=list_df_OR[2], x='C/R/C*R', y=plotting_variable)
# # sns.lineplot(ax=ax, data=list_df_OR[3], x='C/R/C*R', y=plotting_variable)

# file_name = './plots/' + plotting_variable + '_spread.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

<IPython.core.display.Javascript object>

/home/natapueyo/miniconda3/envs/wall_2020/lib/python3.10/site-packages/seaborn/categorical.py:3544: UserWarning: 16.7% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/home/natapueyo/miniconda3/envs/wall_2020/lib/python3.10/site-packages/seaborn/categorical.py:3544: UserWarning: 25.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/home/natapueyo/miniconda3/envs/wall_2020/lib/python3.10/site-packages/seaborn/categorical.py:3544: UserWarning: 17.7% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


Saving plot of FPT and FPE vs density in ./plots/OR_FPT_spread.pdf


/home/natapueyo/miniconda3/envs/wall_2020/lib/python3.10/site-packages/seaborn/categorical.py:3544: UserWarning: 24.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


In [21]:
# # Use seaborn to plot the mean OR-FPE and OR-FPT

# # columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V', 'run_time', 'OR_FPE',
# #                   'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d', 'D_size', 'algorithm', 'C/R/C*R']
# # plotting_variable = 'SW_FPE'

# df_SW_concat = pd.concat([list_df_SW[0], list_df_SW[1], list_df_SW[2], list_df_SW[3]], ignore_index=True)

# sns.set_style("whitegrid")

# x_var   = 'N'

# y1 = 'SW_FPE'
# y2 = 'SW_FPT'
# y3 = 'V'
# y4 = 'N'

# hue_var = 'D_size'
 
# # sns.relplot(
# #     data=df_OR,
# #     x="file", y=plotting_variable,
# #     hue="D_size", style="N", s=100)

# # ax0 = sns.relplot(
# #     data=df_SW_concat, x=x_var, y=y1,
# #     row="C/R/C*R", hue=hue_var, style=hue_var,
# #     kind="scatter"
# # )

# #define plotting region (4 rows, 1 columns)
# fig, axs = plt.subplots(4, 1, figsize=(7,7))



# # # #create boxplot in each subplot
# ax0 = sns.scatterplot(data=df_SW_concat, x=x_var, y=y1, s=5, hue=hue_var, ax=axs[0])
# ax1 = sns.scatterplot(data=df_SW_concat, x=x_var, y=y2, s=5, hue=hue_var, ax=axs[1], legend=False)
# ax2 = sns.scatterplot(data=df_SW_concat, x=x_var, y=y3, s=5, hue=hue_var, ax=axs[2], legend=False)
# ax3 = sns.scatterplot(data=df_SW_concat, x=x_var, y=y4, s=10, hue=hue_var, ax=axs[3],legend=False)

# # ax00 = sns.lineplot(data=df_SW_concat, x=x_var, y=y1, hue=hue_var, ax=axs[0], legend=False)
# # ax11 = sns.lineplot(data=df_SW_concat, x=x_var, y=y2, hue=hue_var, ax=axs[1], legend=False)
# # ax22 = sns.lineplot(data=df_SW_concat, x=x_var, y=y3, hue=hue_var, ax=axs[2], legend=False)
# # ax33 = sns.lineplot(data=df_SW_concat, x=x_var, y=y4, hue=hue_var, ax=axs[3], legend=False)

# # # ax = sns.catplot(data=df_SW_concat, x='C/R/C*R', y=plotting_variable, kind='swarm', hue='D_size', s=5)

# # # ax = sns.catplot(data=df_SW_concat, x='segment_start', y=plotting_variable, kind='swarm', hue='D_size', style='segment', s=3)
# # # ax = sns.lineplot(data=df_SW_concat, x=x_var, y=plotting_variable, hue='C/R/C*R')
# # # sns.scatterplot(data=tips, x=x_var, y="tip", hue="time")

# if x_var == 'segment_start':
#     x_label_str = 'Sliding Window start (m)'
    
# elif x_var == 'C/R/C*R':
#     x_label_str = 'no. columns/rows/arms'
    
# elif x_var == 'N':
#     x_label_str = 'Fruits in SW'

# # # set the labels
# # if plotting_variable == 'OR_FPE':
# #     y_label_str = 'Orchard Row FPE (%)'
    
# # elif plotting_variable == 'OR_FPT':
# #     y_label_str = 'Orchard Row FPT (fruits/s)'
    
# # elif plotting_variable == 'SW_FPT':
# #     y_label_str = 'Sliding Window FPT (fruits/s)'
    
# # elif plotting_variable == 'SW_FPE':
# #     y_label_str = 'Sliding Window FPE (%)'


# ax0.set(xlabel=None, ylabel='SW-FPE\n(%)')
# ax1.set(xlabel=None, ylabel='SW-FPT\n(fruits/s)')
# ax2.set(xlabel=x_label_str, ylabel='Harvester Speed\n(cm/s)')
# ax3.set(xlabel=x_label_str, ylabel='Fruits in SW')

# # fig.tight_layout()

# file_name = './plots/sw_all_n.pdf'
# print('Saving plot of FPT and FPE vs density in', file_name)
# plt.savefig(file_name,dpi=300, bbox_inches='tight')

# plt.show()

<IPython.core.display.Javascript object>

Saving plot of FPT and FPE vs density in ./plots/sw_all_n.pdf


## Divide the plots by the Harvester Configuration

In [46]:
# Use seaborn to plot the mean OR-FPE and OR-FPT

# columns = ['file', 'segment', 'n_segments', 'N', 'N_deadspace', 'Col', 'Row', 'V', 'run_time', 'OR_FPE',
#                   'SW_FPE', 'OR_FPT', 'SW_FPT', 'T_d', 'D_size', 'algorithm', 'C/R/C*R']
df_SW_concat = pd.concat([list_df_SW[0], list_df_SW[1], list_df_SW[2], list_df_SW[3]], ignore_index=True)

sns.set_style("whitegrid")

x_var   = 'N'

y1 = 'SW_FPE'
y2 = 'SW_FPT'
y3 = 'V'

plotting_variable = y3

hue_var = 'D_size'
row_var = 'C/R/C*R'


if x_var == 'segment_start':
    x_label_str = 'Sliding Window start (m)'
elif x_var == 'C/R/C*R':
    x_label_str = 'no. columns/rows/arms'
elif x_var == 'N':
    x_label_str = 'Fruits in SW'

# set the labels
if plotting_variable == 'V':
    y_label_str = 'Harvester Speed\n(cm/s)'
elif plotting_variable == 'OR_FPT':
    y_label_str = 'Orchard Row FPT (fruits/s)'
elif plotting_variable == 'SW_FPT':
    y_label_str = 'SW-FPT\n(fruits/s)'
elif plotting_variable == 'SW_FPE':
    y_label_str = 'SW-FPE\n(%)'


ax0 = sns.relplot(data=df_SW_concat, x=x_var, y=plotting_variable, hue=hue_var, style=hue_var, row=row_var)
ax0.set(xlabel=x_label_str, ylabel=y_label_str)

# could change the legend
# new_title = hue_var
# ax0._legend.set_title(new_title)
# # replace labels
# new_labels = ['1*dw', '1/2*dw', '1/3*dw hallo']
# for t, l in zip(ax0._legend.texts, new_labels):
#     t.set_text(l)

fig.tight_layout()

file_name = './plots/sw_v_n.pdf'
print('Saving plot of FPT and FPE vs density in', file_name)
plt.savefig(file_name,dpi=300, bbox_inches='tight')

plt.show()

<IPython.core.display.Javascript object>

Saving plot of FPT and FPE vs density in ./plots/sw_v_n.pdf
